In [1]:
import os
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
import datetime as dt
import pandas_ta as ta
import yfinance as yf
import copy
import random
from pymongo import MongoClient
import sys
from dotenv import load_dotenv
import warnings

warnings.filterwarnings("ignore")
warnings.simplefilter('ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

load_dotenv()
mongo_client = MongoClient(os.environ.get("MONGO_URI"))
stock_db = mongo_client["stock_db"]
ref_db = mongo_client["ref_db"]

### Chuẩn bị các dữ liệu

##### Các hàm giải mã và lấy dữ liệu

- Lấy dữ liệu từ MongoDB

In [2]:
def get_mongo_df(db_collection, df_name, find_query=None, projection=None):
    # Truy cập collection
    collection = db_collection[df_name]
    # Nếu không truyền vào find_query thì mặc định lấy tất cả document
    if find_query is None:
        find_query = {}
    # Nếu không truyền vào projection thì mặc định loại bỏ trường _id
    if projection is None:
        projection = {"_id": 0}
    # Thực hiện lệnh find với điều kiện và projection đã cho
    docs = collection.find(find_query, projection)
    # Chuyển đổi kết quả sang DataFrame và trả về
    df = pd.DataFrame(list(docs))
    return df

def overwrite_mongo(collection, df):
    # Lấy tên collection hiện tại và database
    collection_name = collection.name
    db = collection.database  # Truy cập database từ collection
    temp_collection_name = f"temp_{collection_name}"
    old_collection_name = f"old_{collection_name}"

    # Reset index của DataFrame
    df = df.reset_index(drop=True)
    records = df.replace({pd.NaT: None}).to_dict(orient='records')

    # 1. Lưu dữ liệu vào collection tạm
    temp_collection = db[temp_collection_name]
    temp_collection.drop()  # Đảm bảo collection tạm sạch trước khi insert
    temp_collection.insert_many(records)

    # 2. Rename collection cũ thành 'old_' (nếu tồn tại)
    if collection_name in db.list_collection_names():
        db[collection_name].rename(old_collection_name, dropTarget=True)

    # 3. Rename collection tạm thành tên chuẩn
    temp_collection.rename(collection_name, dropTarget=True)

    # 4. Xóa collection 'old_' (nếu tồn tại)
    if old_collection_name in db.list_collection_names():
        db[old_collection_name].drop()

- Lấy ra tên quý hiện tại hoặc quý trước đó

In [3]:
def get_quarter(name):
    now = datetime.now()
    year = now.year
    month = now.month
    if 1 <= month <= 3:
        quarter = "q1"
        previous_quarter = "q4"
        previous_year = year - 1
    elif 4 <= month <= 6:
        quarter = "q2"
        previous_quarter = "q1"
        previous_year = year
    elif 7 <= month <= 9:
        quarter = "q3"
        previous_quarter = "q2"
        previous_year = year
    else:
        quarter = "q4"
        previous_quarter = "q3"
        previous_year = year
    
    if name == 'current_quarter':
        return f'{quarter}_{year}'
    if name == 'previous_quarter':
        return f'{previous_quarter}_{previous_year}'

- Giải mã dữ liệu từ file .dat

In [4]:
def decode_data(file_path):
    # Đọc dữ liệu vào một numpy array
    data = np.fromfile(file_path, dtype=np.uint8)

    # Giả định kích thước mỗi bản ghi (có thể thay đổi tùy theo cấu trúc tệp thực tế)
    record_size = 32  # Giả định
    num_records = len(data) // record_size

    # Số lượng cột dữ liệu (bao gồm ngày, thời gian và các giá trị int32 còn lại)
    num_columns = record_size // 4  # Mỗi giá trị int32 chiếm 4 byte

    # Sử dụng numpy để cắt và giải mã dữ liệu hiệu quả hơn
    # Tạo một numpy array để chứa các giá trị int32 và float32
    raw_data = data.reshape(num_records, record_size // 4, 4)

    # Giải mã ngày và thời gian (int32) ở cột 0 và 1, các cột còn lại là float32
    int_data = raw_data[:, :2].view(np.int32)  # Giải mã int32 (2 cột)
    float_data = raw_data[:, 2:].view(np.float32)  # Giải mã float32 (các cột còn lại)

    # Kết hợp dữ liệu
    records = np.hstack((int_data, float_data))

    # Đảm bảo rằng dữ liệu là 2D
    records = records.reshape(num_records, num_columns)

    # Đảo ngược lại dữ liệu trước khi chuyển thành DataFrame
    records = records[::-1]

    # Chuyển đổi thành DataFrame và loại bỏ dòng đầu tiên
    df = pd.DataFrame(records, columns=[f"Col_{i}" for i in range(num_columns)])
    return df  # Loại bỏ đi dòng dữ liệu đầu tiên không cần thiết


- Làm sạch dữ liệu sau khi giải mã

In [5]:
def clean_stock_data(df_raw):
    #Lọc ra ra dữ liệu từ năm 2020
    df_raw = df_raw[df_raw['Col_0'] > 20200400]
    #Tạo cột cap cho cổ phiếu
    df_raw['cap'] = (df_raw['Col_5'] * df_raw['Col_7'])/1000000
    #Xoá đi các cột khong sử dụng
    df_clean = df_raw.drop(columns=['Col_1', 'Col_7'])
    #Chuyển đổi định dạng dữ liệu dang datetime
    df_clean['Col_0'] = pd.to_datetime(df_clean['Col_0'], format='%Y%m%d')
    #Đổi tên cột cho đúng
    df_clean.columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'cap']
    df_clean = df_clean.drop_duplicates(subset=['date'], keep='first')

    return df_clean.reset_index(drop=True)

def clean_index_data(df_raw):
    #Lọc ra ra dữ liệu từ năm 2020
    df_raw = df_raw[df_raw['Col_0'] > 20200400]
    #Xoá đi các cột khong sử dụng
    df_clean = df_raw.drop(columns=['Col_1'])
    #Chuyển đổi định dạng dữ liệu dang datetime
    df_clean['Col_0'] = pd.to_datetime(df_clean['Col_0'], format='%Y%m%d')
    #Đổi tên cột cho đúng
    df_clean.columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'option']
    df_clean = df_clean.drop_duplicates(subset=['date'], keep='first')
    #Điều chỉnh lại giá trị các cột
    df_clean['option'] = df_clean['option']/1000000000
		#Thêm cột phân loại index
    df_clean.insert(0, 'type', 'spot')
    
    return df_clean.reset_index(drop=True)

def clean_futures_data(df_raw):
    #Lọc ra ra dữ liệu từ năm 2020
    df_raw = df_raw[df_raw['Col_0'] > 20200400]
    #Xoá đi các cột khong sử dụng
    df_clean = df_raw.drop(columns=['Col_1'])
    #Chuyển đổi định dạng dữ liệu dang datetime
    df_clean['Col_0'] = pd.to_datetime(df_clean['Col_0'], format='%Y%m%d')
    #Đổi tên cột cho đúng
    df_clean.columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'option']
    df_clean = df_clean.drop_duplicates(subset=['date'], keep='first')
		#Thêm cột phân loại index
    df_clean.insert(0, 'type', 'futures')
    
    return df_clean.reset_index(drop=True)

def clean_itd_data(df_raw):
    #Lọc ra đúng 1 ngày dữ liệu cuối cùng
    df_raw = df_raw[df_raw['Col_0'] == max(df_raw['Col_0'])]
    #Tạo cột date-time mới từ 2 cột date và time cũ
    df_raw['date'] = df_raw['Col_0'].astype(int).astype(str) + ' ' + df_raw['Col_1'].astype(int).astype(str)
    #Xoá đi các cột khong sử dụng
    df_clean = df_raw.drop(columns=['Col_0', 'Col_1', 'Col_7'])
    #Sắp xếp lại thứ tự các cột
    df_clean = df_clean[['date'] + [f"Col_{i}" for i in range(2, len(df_clean.columns)+1)]]
    # #Chuyển đổi định dạng dữ liệu dang datetime
    df_clean['date'] = pd.to_datetime(df_clean['date'], format='%Y%m%d %H%M%S')
    # #Đổi tên cột cho đúng
    df_clean.columns = ['date', 'open', 'high', 'low', 'close', 'volume']
    df_clean = df_clean.drop_duplicates(subset=['date'], keep='first')
    #Làm tròn khung thời gian tới 5 phút
    df_clean['date'] = df_clean['date'].dt.floor('1T')
    df_clean = df_clean.set_index("date").resample("1T", closed='right', label='right').agg({    
        "open": "first",  
        "high": "max",  
        "low": "min", 
        "close": "last",  
        "volume": "sum"   
    }).dropna().reset_index()

    return df_clean.sort_values(by="date", ascending=False).reset_index(drop=True)


- Lấy và làm sạch các dữ liệu khác

In [6]:
def clean_nntd_index_data(df_raw, ticker):
    #Lọc ra ra dữ liệu từ năm 2020
    df_raw = df_raw[df_raw['Col_0'] > 20200400]
    #Xoá đi các cột khong sử dụng
    df_clean = df_raw.drop(columns=['Col_1', 'Col_3', 'Col_4'])
    #Chuyển đổi định dạng dữ liệu dang datetime
    df_clean['Col_0'] = pd.to_datetime(df_clean['Col_0'], format='%Y%m%d')
    #Đổi tên cột cho đúng
    df_clean.columns = ['date', 'sell_volume', 'buy_volume', 'sell_value', 'buy_value']
    df_clean = df_clean.drop_duplicates(subset=['date'], keep='first')
    #Điều chỉnh lại giá trị các cột
    df_clean['buy_volume'] = df_clean['buy_volume']
    df_clean['sell_volume'] = -df_clean['sell_volume']
    df_clean['buy_value'] = df_clean['buy_value']/1000000000
    df_clean['sell_value'] = -df_clean['sell_value']/1000000000
    df_clean['net_volume'] = df_clean['buy_volume'] + df_clean['sell_volume']
    df_clean['net_value'] = df_clean['buy_value'] + df_clean['sell_value']

    if ticker in ['VN30F1M_NN', 'VN30F1M_TD', 'VN30F1Q_NN', 'VN30F1Q_TD', 'VN30F2M_NN', 'VN30F2M_TD', 'VN30F2Q_NN', 'VN30F2Q_TD']:
        df_clean['type'] = 'futures'
    else:
        df_clean['type'] = 'spot'

    df_clean['ticker'] = ticker

    return df_clean.reset_index(drop=True)

def clean_nntd_stock_data(df_raw):
    #Lọc ra ra dữ liệu từ năm 2020
    df_raw = df_raw[df_raw['Col_0'] > 20200400]
    #Xoá đi các cột khong sử dụng
    df_clean = df_raw.drop(columns=['Col_1', 'Col_3', 'Col_4'])
    #Chuyển đổi định dạng dữ liệu dang datetime
    df_clean['Col_0'] = pd.to_datetime(df_clean['Col_0'], format='%Y%m%d')
    #Đổi tên cột cho đúng
    df_clean.columns = ['date', 'sell_volume', 'buy_volume', 'sell_value', 'buy_value']
    df_clean = df_clean.drop_duplicates(subset=['date'], keep='first')
    #Điều chỉnh lại giá trị các cột
    df_clean['buy_volume'] = df_clean['buy_volume']
    df_clean['sell_volume'] = -df_clean['sell_volume']
    df_clean['buy_value'] = df_clean['buy_value']/1000000000
    df_clean['sell_value'] = -df_clean['sell_value']/1000000000
    df_clean['net_volume'] = df_clean['buy_volume'] + df_clean['sell_volume']
    df_clean['net_value'] = df_clean['buy_value'] + df_clean['sell_value']
    
    return df_clean.reset_index(drop=True)

def clean_other_data(df_raw, type):
    #Lọc ra ra dữ liệu từ năm 2020
    df_raw = df_raw[df_raw['Col_0'] > 20200400]
    #Xoá đi các cột khong sử dụng
    df_clean = df_raw.drop(columns=['Col_1'])
    #Chuyển đổi định dạng dữ liệu dang datetime
    df_clean['Col_0'] = pd.to_datetime(df_clean['Col_0'], format='%Y%m%d')
    #Đổi tên cột cho đúng
    df_clean.columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'option']
    df_clean = df_clean.drop_duplicates(subset=['date'], keep='first')
		#Thêm cột phân loại index
    df_clean.insert(0, 'type', type)
    
    return df_clean.reset_index(drop=True)

def get_yfinance_df(ticker, symbol, type_name, today):
	temp_df = yf.download(symbol, start="2020-04-01", end=today.strftime('%Y-%m-%d'), progress=False, auto_adjust=True).reset_index()
	temp_df.columns = temp_df.columns.droplevel(1)
	temp_df.columns.name = None
	temp_df = temp_df.reset_index(drop=True)
	temp_df.columns = ['date','open','high','low','close','volume']
	temp_df['ticker'] = ticker
	temp_df['type'] = type_name
	return temp_df[['ticker','type','date','open','high','low','close','volume']]

- Lấy danh sách và đường dẫn thư mục .dat

In [7]:
def get_file_name_list(folder_path):
    file_name_list = []
    files = os.listdir(folder_path)
    for file in files:
        file_name_list.append(file[:-4])
    return file_name_list

def filter_market_file_name_list(file_name_list):
    filtered_list = [item for item in file_name_list if not (item.endswith('_AC') or item.endswith('_CC'))]
    return filtered_list

eod_stock_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\stock"
eod_index_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\index"
eod_futures_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\futures"
itd_stock_folder_path = "D:\\fireant_metakit\\AmiBroker\\Intraday\\stock"
itd_index_folder_path = "D:\\fireant_metakit\\AmiBroker\\Intraday\\index"
itd_futures_folder_path = "D:\\fireant_metakit\\AmiBroker\\Intraday\\futures"
nn_stock_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\foreign"
td_stock_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\prop"
nntd_index_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\market"
other_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\other"

##### Đọc và tạo các dữ liệu để tham chiếu

- Các biến dùng cho việc maping tên và danh sách tên

In [8]:
#Đọc name map để chuyển đỏi các tên thành dạng full
name_map = get_mongo_df(ref_db, "name_map")
name_map_dict = name_map.set_index('code')['full_name'].to_dict()

order_map = get_mongo_df(ref_db, "order_map")
order_map_dict = order_map.set_index('code')['order'].to_dict()

group_map = get_mongo_df(ref_db, "group_map")
group_map_dict = group_map.set_index('code')['group'].to_dict()

#Tạo các danh sách nhóm trong mỗi cách chia cổ phiếu
all_stock_key_list = [key for key, value in group_map_dict.items() if value == 'all']
industry_name_list = [key for key, value in group_map_dict.items() if value in ['hsA', 'hsB', 'hsC', 'hsD']]
industry_perform_list = [key for key, value in group_map_dict.items() if value == 'hs']
marketcap_group_list = [key for key, value in group_map_dict.items() if value == 'cap']

#Tạo danh danh key cho tổng tất cả các nhóm
group_stock_key_list = all_stock_key_list + industry_name_list + industry_perform_list + marketcap_group_list

- Các biến liên quan tới định nghĩa các quý

In [9]:
#Tạo dict map thời gian và số lượng cổ phiếu
quarter_map = get_mongo_df(ref_db, 'quarter_map')
quarter_map_dict = quarter_map.set_index('quarter').apply(lambda row: row.tolist(), axis=1).to_dict()
quarter_name_list = list(quarter_map_dict.keys())


#Lấy ra list cổ phiếu của giai đoạn hiện tại
quarter_stock_map = get_mongo_df(ref_db, 'quarter_stock_map')
total_stock_list = get_mongo_df(ref_db, 'current_total_stock_list')['ticker'].tolist()
current_quarter_stock_list = list(set(get_file_name_list(itd_stock_folder_path)) 
                                & set(quarter_stock_map[get_quarter('current_quarter')].dropna().tolist()))

#Lấy ra khoảng thời gian tính toán cho quý này và quý trước
calculate_time_span = [quarter_map_dict['q2_2020'][0], quarter_map_dict[get_quarter('current_quarter')][1]]
current_quarter_span = [quarter_map_dict[get_quarter('current_quarter')][0], quarter_map_dict[get_quarter('current_quarter')][1]]
previous_quarter_span = [quarter_map_dict[get_quarter('previous_quarter')][0], quarter_map_dict[get_quarter('previous_quarter')][1]]

- Danh sách phân loại cổ phiếu

In [10]:
#Lấy danh sách phân loại cổ phiếu
stock_classification_df = get_mongo_df(ref_db, 'full_stock_classification')
stock_classification_df = stock_classification_df[stock_classification_df['ticker'].isin(total_stock_list)].reset_index(drop=True)

- Dict chứa ngày bắt đầu và kết thúc từng quý

In [11]:
# Tạo dict chứa ngày bắt đầu và kết thúc của từng quý
quarter_timestamp_map_dict = {}
for key, value in quarter_map_dict.items():
    quarter_timestamp_map_dict[key] = (pd.Timestamp(value[0]), pd.Timestamp(value[1]), value[2])

### Phần dữ liệu lịch sử cổ phiếu

#### Định nghĩa các ticker_dict

In [12]:
#Đợc dữ liệu ticker
index_dict = {}
for ticker in get_file_name_list(eod_index_folder_path):
    temp_file_path = eod_index_folder_path + f'\\{ticker}.dat'
    temp_df_raw = decode_data(temp_file_path)
    temp_df_clean = clean_index_data(temp_df_raw)
    temp_df_clean.insert(0, 'ticker', ticker)
    index_dict[ticker] = temp_df_clean
for ticker in get_file_name_list(eod_futures_folder_path):
    temp_file_path = eod_futures_folder_path + f'\\{ticker}.dat'
    temp_df_raw = decode_data(temp_file_path)
    temp_df_clean = clean_futures_data(temp_df_raw)
    temp_df_clean.insert(0, 'ticker', ticker)
    index_dict[ticker] = temp_df_clean

#Khởi tạo vnindex_series để xác định ngày hiện tại
date_series = pd.DataFrame(index_dict['VNINDEX'].sort_values('date', ascending=False).reset_index(drop=True)['date']).rename(columns={0:'date'})
date_series = date_series[(date_series['date'] >= calculate_time_span[0]) & (date_series['date'] <= calculate_time_span[1])]
today = date_series.iloc[0].item()

In [13]:
# Đọc dữ liệu cổ phiếu
stock_dict = {}
for ticker in total_stock_list:
    if ticker not in get_file_name_list(eod_stock_folder_path):
        pass
    else:
        temp_file_path = eod_stock_folder_path + f'\\{ticker}.dat'
        temp_df_raw = decode_data(temp_file_path)
        temp_df_clean = clean_stock_data(temp_df_raw)
        temp_df_clean.insert(0, 'ticker', ticker)
        stock_dict[ticker] = temp_df_clean

# Bù dữ liệu vào những ngày cổ phiếu không có giao dịch
for ticker, df in stock_dict.items():
    temp_df = date_series.merge(df, on='date', how='left')
    temp_df[['open','high','low','close','cap']] = temp_df[['open','high','low','close','cap']].bfill()
    temp_df['ticker'] = temp_df['ticker'].fillna(ticker)
    temp_df['volume'] = temp_df['volume'].fillna(0)
    stock_dict[ticker] = temp_df

# Thêm cột quarter và cột số lượng cổ phiếu cho từng quý
for ticker, df in stock_dict.items():
    df['quarter'] = df['date'].apply(lambda x: next((key for key, value in quarter_timestamp_map_dict.items() if value[0] <= x <= value[1]), None))
    df['stock_count'] = df['quarter'].apply(lambda x: quarter_map_dict[x][2])

#Tính toán các đường trung bình và các đường MA
stock_dict = {k: v.sort_values(by=['date'], ascending=True).reset_index(drop=True) for k, v in stock_dict.items()}
stock_dict = {
    key: df.assign(
        high5=df['high'].rolling(window=5, min_periods=1).max(),
        low5=df['low'].rolling(window=5, min_periods=1).min(),
        high20=df['high'].rolling(window=20, min_periods=1).max(),
        low20=df['low'].rolling(window=20, min_periods=1).min(),
        high60=df['high'].rolling(window=60, min_periods=1).max(),
        low60=df['low'].rolling(window=60, min_periods=1).min(),
        high120=df['high'].rolling(window=120, min_periods=1).max(),
        low120=df['low'].rolling(window=120, min_periods=1).min(),
        high240=df['high'].rolling(window=240, min_periods=1).max(),
        low240=df['low'].rolling(window=240, min_periods=1).min(),
        high480=df['high'].rolling(window=480, min_periods=1).max(),
        low480=df['low'].rolling(window=480, min_periods=1).min(),

        ma5=df['close'].rolling(window=5, min_periods=1).mean(),
        ma5_V=df['volume'].rolling(window=5, min_periods=1).mean().shift(1),
    )
    for key, df in stock_dict.items()
}
stock_dict = {
    key: df.assign(
        trend_5p=(df['close'] > ((df['high5'] + df['low5'])/2).shift(1)).astype(int),
        trend_20p=(df['close'] > ((df['high20'] + df['low20'])/2).shift(1)).astype(int),
        trend_60p=(df['close'] > ((df['high60'] + df['low60'])/2).shift(1)).astype(int),
        trend_120p=(df['close'] > ((df['high120'] + df['low120'])/2).shift(1)).astype(int),
        trend_240p=(df['close'] > ((df['high240'] + df['low240'])/2).shift(1)).astype(int),
        trend_480p=(df['close'] > ((df['high480'] + df['low480'])/2).shift(1)).astype(int),

        vol_ratio=np.where(df['ma5_V'] != 0, df['volume'] / df['ma5_V'], 0)
    )
    for key, df in stock_dict.items()
}
stock_dict = {k: v[(v['date'] >= calculate_time_span[0]) & (v['date'] <= calculate_time_span[1])].sort_values(by=['date'], ascending=False).reset_index(drop=True) for k, v in stock_dict.items()}

#### Tạo dict chứa dữ liệu của từng quý

In [14]:
#Dict chứa dữ liệu cổ phiếu cho từng giai đoán
quarter_stock_dict = {}
for quarter in quarter_stock_map.columns[1:].tolist():
    quarter_stock_dict[quarter] = {k: v[(v['date'] >= quarter_map_dict[quarter][0]) & 
                                      (v['date'] <= quarter_map_dict[quarter][1])].reset_index(drop=True)
                                      for k, v in copy.deepcopy(stock_dict).items()
                                      if k in quarter_stock_map[quarter].dropna().tolist()}
    
#Dict chưa date series cho từng giai đoạn
quarter_date_series_dict = {}
for quarter in quarter_stock_map.columns[1:].tolist():
    quarter_date_series_dict[quarter] = date_series[(date_series['date'] >= quarter_map_dict[quarter][0]) & 
                                                       (date_series['date'] <= quarter_map_dict[quarter][1])].reset_index(drop=True)

In [15]:
quarter_stock_classification_dict = {}
for quarter in quarter_stock_map.columns[1:].tolist():
    quarter_stock_classification_dict[quarter] = stock_classification_df[stock_classification_df['ticker'].isin(quarter_stock_map[quarter].dropna().tolist())].reset_index(drop=True)
    quarter_stock_classification_dict[quarter] = quarter_stock_classification_dict[quarter][quarter_stock_classification_dict[quarter]['ticker'].isin(stock_dict.keys())]

    price_arr = []
    cap_arr = []
    for ticker in quarter_stock_classification_dict[quarter]['ticker']:
        df = quarter_stock_dict[quarter][ticker]
        if len(df) > 0:
            price_arr.append(df['close'].iloc[-1].item())
            cap_arr.append(df['cap'].iloc[-1].item())
        else:
            price_arr.append(0)
            cap_arr.append(0)

    vonhoa_classification_df = quarter_stock_classification_dict[quarter].copy()
    vonhoa_classification_df['price'] = price_arr
    vonhoa_classification_df['cap'] = cap_arr

    cap_coef = sum(cap_arr)/10000
    vonhoa_classification_df['marketcap_group'] = vonhoa_classification_df.apply(lambda x:
        'small' if ((x['cap']>cap_coef) & (x['cap']<10*cap_coef)) | 
                ((x['cap']>=10*cap_coef) & (x['cap']<20*cap_coef) & (x['price']<10)) 
                else
        ('mid' if ((x['cap']>=10*cap_coef) & (x['cap']<20*cap_coef) & (x['price']>=10)) | 
                ((x['cap']>=20*cap_coef) & (x['cap']<100*cap_coef))
                else
        ('large' if x['cap']>=100*cap_coef
                else 'penny'
    )), axis=1)

    quarter_stock_classification_dict[quarter] = pd.concat([quarter_stock_classification_dict[quarter], vonhoa_classification_df['marketcap_group']], axis=1)


In [16]:
quarter_all_stock = {}
quarter_industry_name = {}
quarter_industry_perform = {}
quarter_marketcap_group = {}

for quarter in quarter_stock_map.columns[1:].tolist():

    quarter_all_stock[quarter] = {}
    quarter_industry_name[quarter] = {}
    quarter_industry_perform[quarter] = {}
    quarter_marketcap_group[quarter] = {}

    stock_by_industry = quarter_stock_classification_dict[quarter].set_index('ticker')['industry_name'].to_dict()
    stock_by_perform = quarter_stock_classification_dict[quarter].set_index('ticker')['industry_perform'].to_dict()
    stock_by_marketcap = quarter_stock_classification_dict[quarter].set_index('ticker')['marketcap_group'].to_dict()

    unique_industries = np.unique(list(stock_by_industry.values()))
    unique_performs = np.unique(list(stock_by_perform.values()))
    unique_marketcaps = ['large', 'mid', 'small', 'penny']

    # Mapping for all
    quarter_all_stock[quarter]['all'] = {key: value for key, value in quarter_stock_dict[quarter].items()}

    # Mapping for industry
    for industry in unique_industries:
        relevant_stocks = [ticker for ticker, ind in stock_by_industry.items() if ind == industry]
        quarter_industry_name[quarter][industry] = {ticker: quarter_stock_dict[quarter][ticker] for ticker in relevant_stocks if ticker in quarter_stock_dict[quarter]}

    # Mapping for performance
    for performance in unique_performs:
        relevant_stocks = [ticker for ticker, perf in stock_by_perform.items() if perf == performance]
        quarter_industry_perform[quarter][performance] = {ticker: quarter_stock_dict[quarter][ticker] for ticker in relevant_stocks if ticker in quarter_stock_dict[quarter]}

    # Mapping for marketcap
    for marketcap in unique_marketcaps:
        relevant_stocks = [ticker for ticker, mcap in stock_by_marketcap.items() if mcap == marketcap]
        quarter_marketcap_group[quarter][marketcap] = {ticker: quarter_stock_dict[quarter][ticker] for ticker in relevant_stocks if ticker in quarter_stock_dict[quarter]}


#### Điểm dòng tiền từng cổ phiếu full lịch sử

In [17]:
def score_calculation(row):
    try:
        # Tính toán giá trị điểm số
        result = (((row['close'] - row['low']) - (row['high'] - row['close'])) / (row['high'] - row['low']) *
                  abs((row['close'] - row['close_prev'])) / row['close_prev'] *
                  (row['volume'] * row['close']) / (row['ma5_prev'] * row['ma5_V'])) * 100 \
                  + ((row['close'] - row['ma5_prev']) / row['ma5_prev']) / 100
        
        # Kiểm tra nếu kết quả là inf, trả về 0 nếu đúng
        if np.isinf(result):
            # Xử lý khi xảy ra lỗi chia cho 0, trả về giá trị tính toán thêm
            return ((row['close'] - row['ma5_prev']) / row['ma5_prev']) / 100
        return result
    except ZeroDivisionError:
        # Xử lý khi xảy ra lỗi chia cho 0, trả về giá trị tính toán thêm
        return ((row['close'] - row['ma5_prev']) / row['ma5_prev']) / 100

In [18]:
#Tính cho toàn bộ lịch sử
raw_stock_score_dict = {}
for ticker in stock_dict.keys():
    #Lọc ra các cột cần sử dụng
    temp_df = stock_dict[ticker][['ticker', 'date', 'quarter', 'stock_count', 'high', 'low', 'close', 'volume', 'vol_ratio', 'ma5', 'ma5_V']]
    #Tính điểm dòng tiền t0 và t5
    temp_df['ma5_prev'] = temp_df['ma5'].shift(-1)
    temp_df['close_prev'] = temp_df['close'].shift(-1)
    temp_df['t0_score'] = temp_df.apply(score_calculation, axis=1)
    temp_df['t5_score'] = temp_df['t0_score'][::-1].rolling(window=5, min_periods=1).mean()[::-1]

    #Gán lại temp_df cho dict
    raw_stock_score_dict[ticker] = temp_df

In [19]:
#Tách điểm dòng tiền vào từng giai đoạn để tính xếp hạng
quarter_stock_score_dict = {}
for quarter in quarter_stock_map.columns[1:].tolist():
    quarter_stock_score_dict[quarter] = {k: v[(v['date'] >= quarter_map_dict[quarter][0]) & 
                                            (v['date'] <= quarter_map_dict[quarter][1])].reset_index(drop=True)
                                            for k, v in copy.deepcopy(raw_stock_score_dict).items()
                                            if k in quarter_stock_map[quarter].dropna().tolist()}

#Tính xếp hạng cổ phiếu cho từng giai đoạn
for quarter in quarter_stock_map.columns[1:].tolist():
    the_dict = quarter_stock_score_dict[quarter]

    #Tính xếp hạng cho cổ phiếu
    t0_ranking_df = quarter_date_series_dict[quarter].copy()
    t5_ranking_df = quarter_date_series_dict[quarter].copy()
    for ticker in the_dict.keys():
        t0_ranking_df[ticker] = the_dict[ticker]['t0_score']
        t0_ranking_df.fillna(0, inplace=True)
        t5_ranking_df[ticker] = the_dict[ticker]['t5_score']
        t5_ranking_df.fillna(0, inplace=True)
    t0_ranking_df = t0_ranking_df.iloc[:,1:].rank(ascending=False, method='min', axis=1)
    t5_ranking_df = t5_ranking_df.iloc[:,1:].rank(ascending=False, method='min', axis=1)

    #Ghép xếp hạng vào bảng thông tin cổ phiếu
    for ticker, df in the_dict.items():
        df['rank_t0'] = t0_ranking_df[ticker]
        df['rank_t5'] = t5_ranking_df[ticker]
        
        #Check xem xếp hạng T0 nằm trong top 10% hay không
        df['top_check'] = df.apply(lambda x: 1 if x['rank_t0'] <= x['stock_count']*0.1 else 0, axis=1)

In [20]:
#Ghép lại bảng điểm dòng tiền cho tất cả giai đoạn
concat_stock_score_dict = {}
for ticker in stock_classification_df['ticker'].tolist():
    temp_df = pd.DataFrame()
    for quarter in quarter_stock_map.columns[1:].tolist():
        if ticker in quarter_stock_map[quarter].tolist():
            temp_df = pd.concat([temp_df, quarter_stock_score_dict[quarter][ticker]], axis=0)

    #Gán lại temp_df cho dict dòng tiền tổng
    concat_stock_score_dict[ticker] = temp_df.reset_index(drop=True)

In [21]:
#Merge bảng và xử lý các giai đoạn không có cổ phiếu
stock_score_dict = {}
for ticker, df in raw_stock_score_dict.items():

    #Nếu trong lịch sử có giai đoạn lọt top thì ghép thêm vào, nếu ko thì thôi
    if (ticker in concat_stock_score_dict) and (len(concat_stock_score_dict[ticker]) > 0):
        df = df.merge(concat_stock_score_dict[ticker][['date','rank_t0','rank_t5','top_check']], on='date', how='left')
    
        df['rank_t0'] = df.apply(lambda x: x['rank_t0'] if pd.notnull(x['rank_t0']) else x['stock_count'], axis=1)
        df['rank_t5'] = df.apply(lambda x: x['rank_t5'] if pd.notnull(x['rank_t5']) else x['stock_count'], axis=1)
        df['top_check'] = df['top_check'].fillna(0)

        #Đếm số phiên lọt top 10%
        df['top_count'] = df['top_check'][::-1].rolling(window=20, min_periods=1).sum()[::-1]

        stock_score_dict[ticker] = df

#### Điểm dòng tiền nhóm cổ phiếu

In [22]:
#Chỉnh sửa lại điểm dòng tiền t0 cho từng cổ phiếu với tác động của độ rộng từng nhóm
def adjust_score_by_breath(t0_score, ratio_column):
    adjusted_score = []
    for score, ratio in zip(t0_score, ratio_column):
        if score >= 0:
            adjusted_score.append(score*ratio)
        else:
            adjusted_score.append(score*(1-ratio))
    return adjusted_score

#Hàm điều chỉnh điểm dòng tiền của cổ phiếu tránh sự đột biến khi đóng góp vào nhóm chung
def adjust_score_for_smooth(row, column_name, max_percent, mark):
    origin_score = row[column_name]
    
    if abs(origin_score) > row['total'] * max_percent:

        sum_abs = row['total'] - abs(row[column_name])
        fixed_score = sum_abs / (1 - max_percent) - sum_abs

        if origin_score >= 0:
            return fixed_score
        else:
            return -fixed_score
    else:
        mark[0] = 0
        return origin_score

#Áp dụng hàm điều chỉnh điểm phía trên vào các nhóm cổ phiếu, việc này lặp lại nhiều lần cho tới khi triệt tiêu sự đột biến
def apply_smooth_score(quarter_score_dict, group_type, quarter):

    #Lấy ra dict điểm dòng tiền và danh sách cổ phiếu của period
    score_dict = quarter_score_dict[quarter]
    quarter_stock_classification_df = quarter_stock_classification_dict[quarter].copy()

    #Lấy ra danh sách cách nhóm nhỏ của mõi cách chia cổ phiếu
    if group_type == 'all':
        key_list = all_stock_key_list
    elif group_type == 'industry_perform':
        key_list = [key for key, value in group_map_dict.items() if value == 'hs']
    elif group_type == 'marketcap_group':
        key_list = [key for key, value in group_map_dict.items() if value == 'cap'] 
    elif group_type == 'industry_name':
        key_list = [key for key, value in group_map_dict.items() if value in ['hsA', 'hsB', 'hsC', 'hsD']]

    for key in key_list:
        score_df = quarter_date_series_dict[quarter].copy()
        if group_type == 'all':
            stock_list = quarter_stock_classification_df['ticker'].tolist()
        else:
            stock_list = [ticker for ticker in 
            quarter_stock_classification_df[quarter_stock_classification_df[group_type]==key]['ticker'].dropna().tolist()]
        for ticker in stock_list:
            try: score_df[ticker] = score_dict[ticker][f't0_score']
            except: pass

        max_percent = max(0.1, min(5*(1/len(stock_list)), 0.5))
        score_df['total'] = score_df.iloc[:, 1:].abs().sum(axis=1)

        mark = [1]
        while True:
            if mark[0] == 1:
                for ticker in stock_list:
                    score_df[ticker] = score_df.iloc[:, 1:].apply(adjust_score_for_smooth, axis=1, args=(ticker, max_percent, mark))
            if mark[0] == 0: break

        for ticker in stock_list:
            try: score_dict[ticker][f't0_{group_type}'] = score_df[ticker]
            except: pass

In [23]:
#Thêm các cột dòng tiền đóng góp vào các nhóm cổ phiếu vào các dict quarter (đã loại bỏ đột biến)
for quarter in quarter_stock_map.columns[1:].tolist():
    for group_type in ['all','industry_name','industry_perform','marketcap_group']:
        apply_smooth_score(quarter_stock_score_dict, group_type, quarter)

In [24]:
#Tính độ rộng dòng tiền các nhóm cổ phiếu từng phiên trong từng giai đoạn
quarter_market_breath_dict = {}
for quarter in quarter_stock_map.columns[1:].tolist():

    quarter_market_breath_dict[quarter] = quarter_date_series_dict[quarter].copy()
    quarter_stock_classification_df = quarter_stock_classification_dict[quarter].copy()
    stock_score_df = quarter_date_series_dict[quarter].copy()

    for ticker, df in quarter_stock_score_dict[quarter].items():
        stock_score_df[ticker] = quarter_stock_score_dict[quarter][ticker]['t0_score']
    stock_score_df.iloc[:,1:] = stock_score_df.iloc[:,1:].applymap(lambda x: 1 if x > 0 else 0)

    all_stock_breadth_dict = {}
    for key in all_stock_key_list:
        stock_list = quarter_stock_classification_df['ticker'].tolist()
        all_stock_breadth_dict[key] = stock_score_df[['date'] + [columns for columns in stock_list]]
        quarter_market_breath_dict[quarter][key] = all_stock_breadth_dict[key].iloc[:,1:].sum(axis=1)/len(stock_list)
    
    industry_name_breadth_dict = {}
    for key in industry_name_list:
        stock_list = quarter_stock_classification_df[quarter_stock_classification_df['industry_name']==key]['ticker'].tolist()
        industry_name_breadth_dict[key] = stock_score_df[['date'] + [columns for columns in stock_list]]
        quarter_market_breath_dict[quarter][key] = industry_name_breadth_dict[key].iloc[:,1:].sum(axis=1)/len(stock_list)

    industry_perform_breadth_dict = {}
    for key in industry_perform_list:
        stock_list = quarter_stock_classification_df[quarter_stock_classification_df['industry_perform']==key]['ticker'].tolist()
        industry_perform_breadth_dict[key] = stock_score_df[['date'] + [columns for columns in stock_list]]
        quarter_market_breath_dict[quarter][key] = industry_perform_breadth_dict[key].iloc[:,1:].sum(axis=1)/len(stock_list)

    marketcap_group_breadth_dict = {}
    for key in marketcap_group_list:
        stock_list = quarter_stock_classification_df[quarter_stock_classification_df['marketcap_group']==key]['ticker'].tolist()
        marketcap_group_breadth_dict[key] = stock_score_df[['date'] + [columns for columns in stock_list]]
        quarter_market_breath_dict[quarter][key] = marketcap_group_breadth_dict[key].iloc[:,1:].sum(axis=1)/len(stock_list)
    
#Chỉnh sửa lại điểm dòng tiền trực tiếp vào trong quarter_stock_score_dict
for quarter in quarter_stock_map.columns[1:].tolist():
    quarter_stock_classification_df = quarter_stock_classification_dict[quarter].copy()
    for ticker, df in quarter_stock_score_dict[quarter].items():
        for group_type in ['all','industry_name','industry_perform','marketcap_group']:
            if group_type == 'all':
                df[f't0_{group_type}'] = adjust_score_by_breath(df[f't0_{group_type}'], quarter_market_breath_dict[quarter]['all'])
            else:
                group_name = quarter_stock_classification_df[quarter_stock_classification_df['ticker']==ticker][group_type].item()
                df[f't0_{group_type}'] = adjust_score_by_breath(df[f't0_{group_type}'], quarter_market_breath_dict[quarter][group_name])

In [25]:
#Tạo dict dòng tiền các nhóm cổ phiếu từng giai đoạn
quarter_group_score_dict = {}
for quarter in quarter_stock_map.columns[1:].tolist():
    quarter_group_score_dict[quarter] = quarter_date_series_dict[quarter].copy()
    quarter_stock_classification_df = quarter_stock_classification_dict[quarter].copy()

    #Thêm cột điểm dòng tiền toàn bộ cổ phiếu
    for key in all_stock_key_list:
        score_df = quarter_date_series_dict[quarter].copy()
        for ticker in quarter_stock_classification_df['ticker']:
            score_df[ticker] = quarter_stock_score_dict[quarter][ticker]['t0_all']
        score_df['total'] = score_df.iloc[:, 1:].mean(axis=1)
        quarter_group_score_dict[quarter][key] = score_df['total']

    #Thêm các cột điểm dòng tiền ngành
    for nganh in industry_name_list:
        score_df = quarter_date_series_dict[quarter].copy()
        for ticker in quarter_stock_classification_df[quarter_stock_classification_df['industry_name']==nganh]['ticker']:
            score_df[ticker] = quarter_stock_score_dict[quarter][ticker]['t0_industry_name']
        score_df['total'] = score_df.iloc[:, 1:].mean(axis=1)
        quarter_group_score_dict[quarter][nganh] = score_df['total']

    #Thêm các cột điểm dòng tiền nhóm hiệu suất
    for group in industry_perform_list:
        score_df = quarter_date_series_dict[quarter].copy()
        for ticker in quarter_stock_classification_df[quarter_stock_classification_df['industry_perform']==group]['ticker']:
            score_df[ticker] = quarter_stock_score_dict[quarter][ticker]['t0_industry_perform']
        score_df['total'] = score_df.iloc[:, 1:].mean(axis=1)
        quarter_group_score_dict[quarter][group] = score_df['total']

    #Thêm các cột điểm dòng tiền nhóm vốn hoá
    for marketcap in marketcap_group_list:
        score_df = quarter_date_series_dict[quarter].copy()
        for ticker in quarter_stock_classification_df[quarter_stock_classification_df['marketcap_group']==marketcap]['ticker']:
            score_df[ticker] = quarter_stock_score_dict[quarter][ticker]['t0_marketcap_group']
        score_df['total'] = score_df.iloc[:, 1:].mean(axis=1)
        quarter_group_score_dict[quarter][marketcap] = score_df['total']

#Ghép các bảng dòng tiền nhóm cổ phiếu từng giai đoạn thành bảng full
group_score_df = pd.DataFrame()
for quarter in quarter_stock_map.columns[1:].tolist():
    group_score_df = pd.concat([group_score_df, quarter_group_score_dict[quarter]]).sort_values('date', ascending=False).reset_index(drop=True)

group_score_df = group_score_df.fillna(0)

In [26]:
def mean_of_positive_values(df, length):
    positive_values = df[df >= 0]  # Lọc ra các giá trị âm
    return positive_values.sum(axis=1)/length

#Tạo dict dòng tiền các nhóm cổ phiếu từng giai đoạn
quarter_group_score_positive_dict = {}
for quarter in quarter_stock_map.columns[1:].tolist():
    quarter_group_score_positive_dict[quarter] = quarter_date_series_dict[quarter].copy()
    quarter_stock_classification_df = quarter_stock_classification_dict[quarter].copy()

    #Thêm cột điểm dòng tiền toàn bộ cổ phiếu
    for key in all_stock_key_list:
        score_df = quarter_date_series_dict[quarter].copy()
        temp_stock_list = quarter_stock_classification_df['ticker']
        for ticker in temp_stock_list:
            score_df[ticker] = quarter_stock_score_dict[quarter][ticker]['t0_all']
        score_df['total'] = mean_of_positive_values(score_df.iloc[:, 1:], len(temp_stock_list))
        quarter_group_score_positive_dict[quarter][key] = score_df['total']

    #Thêm các cột điểm dòng tiền ngành
    for nganh in industry_name_list:
        score_df = quarter_date_series_dict[quarter].copy()
        temp_stock_list = quarter_stock_classification_df[quarter_stock_classification_df['industry_name']==nganh]['ticker']
        for ticker in temp_stock_list:
            score_df[ticker] = quarter_stock_score_dict[quarter][ticker]['t0_industry_name']
        score_df['total'] = mean_of_positive_values(score_df.iloc[:, 1:], len(temp_stock_list))
        quarter_group_score_positive_dict[quarter][nganh] = score_df['total']

    #Thêm các cột điểm dòng tiền nhóm hiệu suất
    for group in industry_perform_list:
        score_df = quarter_date_series_dict[quarter].copy()
        temp_stock_list = quarter_stock_classification_df[quarter_stock_classification_df['industry_perform']==group]['ticker']
        for ticker in temp_stock_list:
            score_df[ticker] = quarter_stock_score_dict[quarter][ticker]['t0_industry_perform']
        score_df['total'] = mean_of_positive_values(score_df.iloc[:, 1:], len(temp_stock_list))
        quarter_group_score_positive_dict[quarter][group] = score_df['total']

    #Thêm các cột điểm dòng tiền nhóm vốn hoá
    for marketcap in marketcap_group_list:
        score_df = quarter_date_series_dict[quarter].copy()
        temp_stock_list = quarter_stock_classification_df[quarter_stock_classification_df['marketcap_group']==marketcap]['ticker']
        for ticker in temp_stock_list:
            score_df[ticker] = quarter_stock_score_dict[quarter][ticker]['t0_marketcap_group']
        score_df['total'] = mean_of_positive_values(score_df.iloc[:, 1:], len(temp_stock_list))
        quarter_group_score_positive_dict[quarter][marketcap] = score_df['total']

#Ghép các bảng dòng tiền nhóm cổ phiếu từng giai đoạn thành bảng full
group_score_positive_df = pd.DataFrame()
for quarter in quarter_stock_map.columns[1:].tolist():
    group_score_positive_df = pd.concat([group_score_positive_df, quarter_group_score_positive_dict[quarter]]).sort_values('date', ascending=False).reset_index(drop=True)

group_score_positive_df = group_score_positive_df.fillna(0)


In [27]:
def mean_of_negative_values(df, length):
    negative_values = df[df < 0]  # Lọc ra các giá trị âm
    return negative_values.sum(axis=1)/length

#Tạo dict dòng tiền các nhóm cổ phiếu từng giai đoạn
quarter_group_score_negative_dict = {}
for quarter in quarter_stock_map.columns[1:].tolist():
    quarter_group_score_negative_dict[quarter] = quarter_date_series_dict[quarter].copy()
    quarter_stock_classification_df = quarter_stock_classification_dict[quarter].copy()

    #Thêm cột điểm dòng tiền toàn bộ cổ phiếu
    for key in all_stock_key_list:
        score_df = quarter_date_series_dict[quarter].copy()
        temp_stock_list = quarter_stock_classification_df['ticker']
        for ticker in temp_stock_list:
            score_df[ticker] = quarter_stock_score_dict[quarter][ticker]['t0_all']
        score_df['total'] = mean_of_negative_values(score_df.iloc[:, 1:], len(temp_stock_list))
        quarter_group_score_negative_dict[quarter][key] = score_df['total']

    #Thêm các cột điểm dòng tiền ngành
    for nganh in industry_name_list:
        score_df = quarter_date_series_dict[quarter].copy()
        temp_stock_list = quarter_stock_classification_df[quarter_stock_classification_df['industry_name']==nganh]['ticker']
        for ticker in temp_stock_list:
            score_df[ticker] = quarter_stock_score_dict[quarter][ticker]['t0_industry_name']
        score_df['total'] = mean_of_negative_values(score_df.iloc[:, 1:], len(temp_stock_list))
        quarter_group_score_negative_dict[quarter][nganh] = score_df['total']

    #Thêm các cột điểm dòng tiền nhóm hiệu suất
    for group in industry_perform_list:
        score_df = quarter_date_series_dict[quarter].copy()
        temp_stock_list = quarter_stock_classification_df[quarter_stock_classification_df['industry_perform']==group]['ticker']
        for ticker in temp_stock_list:
            score_df[ticker] = quarter_stock_score_dict[quarter][ticker]['t0_industry_perform']
        score_df['total'] = mean_of_negative_values(score_df.iloc[:, 1:], len(temp_stock_list))
        quarter_group_score_negative_dict[quarter][group] = score_df['total']

    #Thêm các cột điểm dòng tiền nhóm vốn hoá
    for marketcap in marketcap_group_list:
        score_df = quarter_date_series_dict[quarter].copy()
        temp_stock_list = quarter_stock_classification_df[quarter_stock_classification_df['marketcap_group']==marketcap]['ticker']
        for ticker in temp_stock_list:
            score_df[ticker] = quarter_stock_score_dict[quarter][ticker]['t0_marketcap_group']
        score_df['total'] = mean_of_negative_values(score_df.iloc[:, 1:], len(temp_stock_list))
        quarter_group_score_negative_dict[quarter][marketcap] = score_df['total']

#Ghép các bảng dòng tiền nhóm cổ phiếu từng giai đoạn thành bảng full
group_score_negative_df = pd.DataFrame()
for quarter in quarter_stock_map.columns[1:].tolist():
    group_score_negative_df = pd.concat([group_score_negative_df, quarter_group_score_negative_dict[quarter]]).sort_values('date', ascending=False).reset_index(drop=True)

group_score_negative_df = group_score_negative_df.fillna(0)

#### Xếp hạng các nhóm cổ phiếu

In [28]:
#Tạo bảng xếp hạng cho các nhóm cổ phiếu
def create_ranking_df(score_df):
    socre_dict = {}
    for group in score_df.columns[1:]:
        socre_dict[group] = date_series.copy()
        socre_dict[group]['t0_score'] = score_df[group]
        socre_dict[group]['t5_score'] = socre_dict[group]['t0_score'][::-1].rolling(window=5).mean()[::-1]

    ranking_score = date_series.copy()
    for group in socre_dict.keys():
        ranking_score[group] = socre_dict[group]['t5_score']
        ranking_score.fillna(0,inplace=True)

    ranking_df = date_series.copy()
    for group in socre_dict.keys():
        ranking_df[group] = 0

    for i in range(len(date_series.copy())):
        ranking_df.iloc[i, 1:] = ranking_score.iloc[i, 1:].rank(ascending=False, method='min')

    return ranking_df

industry_name_ranking_df = create_ranking_df(group_score_df[['date'] + industry_name_list])
industry_perform_ranking_df = create_ranking_df(group_score_df[['date'] + industry_perform_list])
marketcap_group_ranking_df = create_ranking_df(group_score_df[['date'] + marketcap_group_list])
group_score_ranking_df = industry_name_ranking_df.merge(industry_perform_ranking_df, on='date', how='left').merge(marketcap_group_ranking_df, on='date', how='left')

#Thêm cột xếp hạng cho nhóm 'all' là 1 để khớp dữ liệu bảng
group_score_ranking_df['all'] = 1

#### Thanh khoản các nhóm cổ phiếu

In [29]:
#Tạo dict chứa thanh khoản trung bình của mỗi nhóm cổ phiếu trong các giai đoạn
quarter_group_volume_dict = {}

for quarter in quarter_stock_map.columns[1:].tolist():

    quarter_group_volume_dict[quarter] = quarter_date_series_dict[quarter].copy()
    quarter_date_series = quarter_date_series_dict[quarter].copy()
    
    for name in all_stock_key_list:
        temp_volume_df = quarter_date_series.copy()
        for ticker, df in quarter_all_stock[quarter][name].items():
            temp_volume_df[ticker] = df['volume']
        temp_volume_df['volume'] = temp_volume_df.iloc[:, 1:].sum(axis=1)
        quarter_group_volume_dict[quarter][name] = temp_volume_df['volume']

    for name in industry_name_list:
        temp_volume_df = quarter_date_series.copy()
        for ticker, df in quarter_industry_name[quarter][name].items():
            temp_volume_df[ticker] = df['volume']
        temp_volume_df['volume'] = temp_volume_df.iloc[:, 1:].sum(axis=1)
        quarter_group_volume_dict[quarter][name] = temp_volume_df['volume']

    for name in industry_perform_list:
        temp_volume_df = quarter_date_series.copy()
        for ticker, df in quarter_industry_perform[quarter][name].items():
            temp_volume_df[ticker] = df['volume']
        temp_volume_df['volume'] = temp_volume_df.iloc[:, 1:].sum(axis=1)
        quarter_group_volume_dict[quarter][name] = temp_volume_df['volume']

    for name in marketcap_group_list:
        temp_volume_df = quarter_date_series.copy()
        for ticker, df in quarter_marketcap_group[quarter][name].items():
            temp_volume_df[ticker] = df['volume']
        temp_volume_df['volume'] = temp_volume_df.iloc[:, 1:].sum(axis=1)
        quarter_group_volume_dict[quarter][name] = temp_volume_df['volume']

#Ghép các bảng dòng tiền nhóm cổ phiếu từng giai đoạn thành bảng full thanh khoản
group_volume_df = pd.DataFrame()
for quarter in quarter_stock_map.columns[1:].tolist():
    group_volume_df = pd.concat([group_volume_df, quarter_group_volume_dict[quarter]]).sort_values('date', ascending=False).reset_index(drop=True)

#Tạo bảng hệ số thanh khoản để lấy phiên cuối
group_vol_ratio_df = date_series.copy()
temp_df = date_series.copy()
for key in group_stock_key_list:
    temp_df[key] = group_volume_df[key][::-1].rolling(window=5).mean()[::-1].shift(-1)
    group_vol_ratio_df[key] = group_volume_df[key]/temp_df[key]
    group_vol_ratio_df[key].iloc[0] = group_volume_df[key].iloc[0]/(temp_df[key].iloc[0])

#### Price Index các nhóm cổ phiếu

In [30]:
def calculate_total_change(stock_group, name, price_index_date_series):
    quarter_index_df = price_index_date_series.copy()

    for ticker, df in stock_group[name].items():
        quarter_index_df[ticker] = df['close']
        quarter_index_df[ticker] = quarter_index_df[ticker][::-1].pct_change()[::-1]

    quarter_index_df['total_change'] = quarter_index_df.iloc[:,1:].sum(axis=1)
    quarter_index_df['total_change'] = ((quarter_index_df['total_change']/len(stock_group[name]))*100)
    quarter_index_df['total_change'] = quarter_index_df['total_change']*10

    return quarter_index_df['total_change']

In [31]:
quarter_group_price_index_dict = {}
for quarter in quarter_stock_map.columns[1:].tolist():

    quarter_date_series = quarter_date_series_dict[quarter].copy()
    temp_df = quarter_date_series.copy()

    for key in all_stock_key_list:
        temp_df[key] = calculate_total_change(quarter_all_stock[quarter], key, quarter_date_series)

    for key in industry_name_list:
        temp_df[key] = calculate_total_change(quarter_industry_name[quarter], key, quarter_date_series)

    for key in industry_perform_list:
        temp_df[key] = calculate_total_change(quarter_industry_perform[quarter], key, quarter_date_series)

    for key in marketcap_group_list:
        temp_df[key] = calculate_total_change(quarter_marketcap_group[quarter], key, quarter_date_series)

    quarter_group_price_index_dict[quarter] = temp_df

group_price_index_df= pd.DataFrame()
for quarter in quarter_stock_map.columns[1:].tolist():
    group_price_index_df = pd.concat([group_price_index_df, quarter_group_price_index_dict[quarter]]).sort_values('date', ascending=False).reset_index(drop=True)

for key in group_stock_key_list:
    group_price_index_df[key] = group_price_index_df[key][::-1].cumsum()[::-1] + 1000

#### Tạo các bảng lịch sử dữ liệu

In [32]:
def calculate_extreme_2period(df, quarter_col, value_col, periods=2, date_col='date'):
    """
    Tính giá trị cực trị (max với 'high' hoặc min với 'low') của cột value_col
    trên các dòng có ngày <= date_col và thuộc các nhóm thời gian từ
    (current quarter và (periods-1) nhóm liền trước), sử dụng các thao tác vectorized.
    
    Tham số:
      - df: DataFrame chứa dữ liệu.
      - quarter_col: tên cột định danh kỳ (ví dụ: 'week', 'month', 'quarter', 'year').
      - value_col: tên cột giá trị cần tính (chỉ hỗ trợ 'high' hoặc 'low').
      - periods: số nhóm cần gộp (mặc định 2: nhóm hiện tại và nhóm liền trước).
      - date_col: tên cột ngày (mặc định 'date').
    
    Hàm trả về Series chứa giá trị cực trị ứng với mỗi dòng.
    """
    df = df.copy()
    # Đảm bảo cột ngày ở dạng datetime và sắp xếp theo ngày
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.sort_values(date_col)
    
    # Xác định thứ tự của các nhóm dựa trên ngày xuất hiện đầu tiên của mỗi nhóm
    quarter_min_dates = df.groupby(quarter_col)[date_col].min().sort_values()
    quarter_order = quarter_min_dates.index.tolist()
    mapping = {p: i for i, p in enumerate(quarter_order)}
    df['quarter_idx'] = df[quarter_col].map(mapping)
    
    # Tính giá trị tích lũy (cum_ext) trong mỗi nhóm: với 'high' dùng cummax, với 'low' dùng cummin
    if value_col == 'high':
        df['cum_ext'] = df.groupby(quarter_col)[value_col].cummax()
    elif value_col == 'low':
        df['cum_ext'] = df.groupby(quarter_col)[value_col].cummin()
    else:
        raise ValueError("value_col phải là 'high' hoặc 'low'")
    
    # Tính giá trị cực trị tổng thể của mỗi nhóm (group_ext)
    if value_col == 'high':
        group_ext = df.groupby(quarter_col)[value_col].max()
    else:  # value_col == 'low'
        group_ext = df.groupby(quarter_col)[value_col].min()
    
    groups_df = group_ext.reset_index().rename(columns={value_col: 'group_ext'})
    groups_df['quarter_idx'] = groups_df[quarter_col].map(mapping)
    groups_df = groups_df.sort_values('quarter_idx')
    
    # Tính giá trị cực trị của các nhóm liền trước, dùng rolling trên các group_ext (không tính group hiện tại)
    if periods > 1:
        if value_col == 'high':
            groups_df['prev_ext'] = groups_df['group_ext'].shift(1).rolling(window=periods-1, min_periods=1).max()
        else:  # value_col == 'low'
            groups_df['prev_ext'] = groups_df['group_ext'].shift(1).rolling(window=periods-1, min_periods=1).min()
    else:
        groups_df['prev_ext'] = pd.NA
    
    # Gộp kết quả của các nhóm (prev_ext) vào DataFrame gốc theo quarter_col
    df = df.merge(groups_df[[quarter_col, 'prev_ext']], on=quarter_col, how='left')
    
    # Với mỗi dòng, kết hợp giá trị tích lũy của nhóm hiện tại (cum_ext) với giá trị cực trị của các nhóm trước (prev_ext)
    if value_col == 'high':
        # Nếu prev_ext là NaN thì chỉ dùng cum_ext, ngược lại lấy max của hai giá trị
        df['extreme'] = df[['cum_ext', 'prev_ext']].max(axis=1)
    else:
        df['extreme'] = df[['cum_ext', 'prev_ext']].min(axis=1)

    return df['extreme']

In [33]:
def add_prev_quarter_metrics(df, quarter_col, date_col='date', prefix=None):
    df = df.copy()
    # Chuyển đổi cột date và sắp xếp theo ngày tăng dần
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.sort_values(date_col)
    
    # Xác định thứ tự các kỳ dựa trên ngày xuất hiện đầu tiên của mỗi kỳ
    quarter_order = df.groupby(quarter_col)[date_col].min().sort_values().index.tolist()
    
    # Tính toán các chỉ số cho từng kỳ
    agg_df = df.groupby(quarter_col).agg(
        close_period = ('close', 'last'),  # giá đóng cửa của ngày cuối cùng trong nhóm
        high_period  = ('high', 'max'),      # giá cao nhất của nhóm
        low_period   = ('low', 'min')        # giá thấp nhất của nhóm
    ).reset_index()
    
    # Đặt cột quarter_col thành kiểu categorical với thứ tự xác định
    agg_df[quarter_col] = pd.Categorical(agg_df[quarter_col], categories=quarter_order, ordered=True)
    agg_df = agg_df.sort_values(quarter_col)
    
    # Dịch chuyển các chỉ số để lấy giá của kỳ liền trước
    agg_df['prev_close'] = agg_df['close_period'].shift(1)
    agg_df['prev_high']  = agg_df['high_period'].shift(1)
    agg_df['prev_low']   = agg_df['low_period'].shift(1)
    
    if prefix:
        agg_df = agg_df.rename(columns={
            'prev_close': f'{prefix}_prev_close',
            'prev_high':  f'{prefix}_prev_high',
            'prev_low':   f'{prefix}_prev_low'
        })
        merge_cols = [quarter_col, f'{prefix}_prev_close', f'{prefix}_prev_high', f'{prefix}_prev_low']
    else:
        merge_cols = [quarter_col, 'prev_close', 'prev_high', 'prev_low']
    
    # Gộp các chỉ số của kỳ liền trước vào DataFrame ban đầu
    df = df.merge(agg_df[merge_cols], on=quarter_col, how='left')
    
    return df

##### Bảng lịch sử biểu đồ MS

In [34]:
def transform_ms(stock_group, quarter):
    stock_dict = copy.deepcopy(stock_group)
    # Prepare a base date DataFrame from date_series
    dates_df = quarter_date_series_dict[quarter]
    
    for group_name, stocks in stock_dict.items():
        # Initialize a DataFrame for group trends
        group_trends = dates_df.copy()

        # Compute trends across stocks
        for trend in ['trend_5p', 'trend_20p', 'trend_60p', 'trend_120p', 'trend_240p', 'trend_480p']:
            # Concatenate all trend data for current trend across all stocks
            trend_data = pd.concat([stocks[ticker][trend] for ticker in stocks], axis=1)
            trend_data.fillna(0, inplace=True)
            
            # Calculate the sum and percent for the trend
            sum_trend = trend_data.sum(axis=1)
            percent_trend = sum_trend / len(stocks)
            
            # Add to group trends DataFrame
            group_trends[f'{trend}'] = percent_trend

        stock_dict[group_name] = group_trends.sort_values('date', ascending=False)
        
    return stock_dict

In [35]:
#Tạo các bảng MS cho từng giai đoạn
quarter_all_stock_ms = {}
quarter_industry_name_ms = {}
quarter_industry_perform_ms = {}
quarter_marketcap_group_ms = {}

for quarter in quarter_stock_map.columns[1:].tolist():
    quarter_all_stock_ms[quarter] = transform_ms(quarter_all_stock[quarter], quarter)
    quarter_industry_name_ms[quarter] = transform_ms(quarter_industry_name[quarter], quarter)
    quarter_industry_perform_ms[quarter] = transform_ms(quarter_industry_perform[quarter], quarter)
    quarter_marketcap_group_ms[quarter] = transform_ms(quarter_marketcap_group[quarter], quarter)

In [36]:
#Ghép các bảng MS vào chung 1 bảng full lịch sử
all_stock_ms = {}
industry_name_ms = {}
industry_perform_ms = {}
marketcap_group_ms = {}

for group in all_stock_key_list:
    all_stock_ms[group] = pd.DataFrame()
    for quarter in quarter_stock_map.columns[1:].tolist():
        all_stock_ms[group] = pd.concat([all_stock_ms[group], quarter_all_stock_ms[quarter][group]])

for group in industry_name_list:
    industry_name_ms[group] = pd.DataFrame()
    for quarter in quarter_stock_map.columns[1:].tolist():
        industry_name_ms[group] = pd.concat([industry_name_ms[group], quarter_industry_name_ms[quarter][group]])
    industry_name_ms[group] = industry_name_ms[group]

for group in industry_perform_list:
    industry_perform_ms[group] = pd.DataFrame()
    for quarter in quarter_stock_map.columns[1:].tolist():
        industry_perform_ms[group] = pd.concat([industry_perform_ms[group], quarter_industry_perform_ms[quarter][group]])
    industry_perform_ms[group] = industry_perform_ms[group]

for group in marketcap_group_list:
    marketcap_group_ms[group] = pd.DataFrame()
    for quarter in quarter_stock_map.columns[1:].tolist():
        marketcap_group_ms[group] = pd.concat([marketcap_group_ms[group], quarter_marketcap_group_ms[quarter][group]])
    marketcap_group_ms[group] = marketcap_group_ms[group]

#Gộp tất cả biểu đồ MS vào 1 bảng
history_ms_chart_df = pd.DataFrame()
for item in [all_stock_ms, industry_name_ms, industry_perform_ms, marketcap_group_ms]:
    for group, df in item.items():
        temp_df = df[df['date'] != today.strftime('%Y-%m-%d')]
        temp_df['ticker'] = group
        history_ms_chart_df = pd.concat([history_ms_chart_df, temp_df], axis=0).sort_values('date', ascending=False)

##### Bảng lịch sử từng cổ phiếu

In [37]:
def merge_remove_duplicates(df1, df2, key):
    # Thực hiện merge với tùy chọn suffixes để phân biệt các cột trùng lặp từ df2
    merged = pd.merge(df1, df2, on=key, how='outer', suffixes=('', '_dup'))
    # Tìm các cột có suffix '_dup' (tức là các cột trùng lặp từ df2)
    dup_cols = [col for col in merged.columns if col.endswith('_dup')]
    # Loại bỏ các cột trùng lặp
    merged.drop(columns=dup_cols, inplace=True)
    return merged

history_stock_dict = {}
for ticker in stock_dict.keys():
# for ticker in ['AAA','ACB']:
	df1 = stock_dict[ticker]
	df2 = stock_score_dict[ticker]

	temp_df = merge_remove_duplicates(df1, df2, 'date').sort_values('date').reset_index(drop=True)
	temp_df = temp_df.drop(columns=['ma5', 'trend_5p', 'trend_20p', 'trend_60p', 'trend_120p', 'trend_240p', 'trend_480p'])

	#Tạo thêm các cột mốc tuần tháng
	temp_df['week'] = temp_df['date'].dt.strftime('%Y-%W').apply(lambda x: f'{int(x[:4])-1}-53' if x[-2:] == '00' else x)
	temp_df['month'] = temp_df['date'].dt.strftime('%Y-%m')
	temp_df['year'] = temp_df['date'].dt.strftime('%Y')

	#Tính toán giá của các nến
	temp_df['week_high'] = temp_df.groupby('week')['high'].cummax()
	temp_df['week_low'] = temp_df.groupby('week')['low'].cummin()
	temp_df['week_open'] = temp_df.groupby('week')['open'].transform('first')     
	temp_df['month_high'] = temp_df.groupby('month')['high'].cummax()
	temp_df['month_low'] = temp_df.groupby('month')['low'].cummin()
	temp_df['month_open'] = temp_df.groupby('month')['open'].transform('first')
	temp_df['quarter_high'] = temp_df.groupby('quarter')['high'].cummax()
	temp_df['quarter_low'] = temp_df.groupby('quarter')['low'].cummin()
	temp_df['quarter_open'] = temp_df.groupby('quarter')['open'].transform('first')
	temp_df['year_high'] = temp_df.groupby('year')['high'].cummax()
	temp_df['year_low'] = temp_df.groupby('year')['low'].cummin()
	temp_df['year_open'] = temp_df.groupby('year')['open'].transform('first')

	#Tính toán cho chỉ báo Fibbonachi
	temp_df['2week_high'] = calculate_extreme_2period(temp_df, 'week', 'high')
	temp_df['2week_low'] = calculate_extreme_2period(temp_df, 'week', 'low')
	temp_df['2month_high'] = calculate_extreme_2period(temp_df, 'month', 'high')
	temp_df['2month_low'] = calculate_extreme_2period(temp_df, 'month', 'low')
	temp_df['2quarter_high'] = calculate_extreme_2period(temp_df, 'quarter', 'high')
	temp_df['2quarter_low'] = calculate_extreme_2period(temp_df, 'quarter', 'low')
	temp_df['2year_high'] = calculate_extreme_2period(temp_df, 'year', 'high')
	temp_df['2year_low'] = calculate_extreme_2period(temp_df, 'year', 'low')
      
	temp_df['WFIBO_0382'] = temp_df['2week_high'] - (temp_df['2week_high']-temp_df['2week_low'])*0.382
	temp_df['WFIBO_0500'] = temp_df['2week_high'] - (temp_df['2week_high']-temp_df['2week_low'])*0.500
	temp_df['WFIBO_0618'] = temp_df['2week_high'] - (temp_df['2week_high']-temp_df['2week_low'])*0.618
	temp_df['MFIBO_0382'] = temp_df['2month_high'] - (temp_df['2month_high']-temp_df['2month_low'])*0.382
	temp_df['MFIBO_0500'] = temp_df['2month_high'] - (temp_df['2month_high']-temp_df['2month_low'])*0.500
	temp_df['MFIBO_0618'] = temp_df['2month_high'] - (temp_df['2month_high']-temp_df['2month_low'])*0.618
	temp_df['QFIBO_0382'] = temp_df['2quarter_high'] - (temp_df['2quarter_high']-temp_df['2quarter_low'])*0.382
	temp_df['QFIBO_0500'] = temp_df['2quarter_high'] - (temp_df['2quarter_high']-temp_df['2quarter_low'])*0.500
	temp_df['QFIBO_0618'] = temp_df['2quarter_high'] - (temp_df['2quarter_high']-temp_df['2quarter_low'])*0.618
	temp_df['YFIBO_0382'] = temp_df['2year_high'] - (temp_df['2year_high']-temp_df['2year_low'])*0.382
	temp_df['YFIBO_0500'] = temp_df['2year_high'] - (temp_df['2year_high']-temp_df['2year_low'])*0.500
	temp_df['YFIBO_0618'] = temp_df['2year_high'] - (temp_df['2year_high']-temp_df['2year_low'])*0.618
  
	#Tính toán cho chỉ báo Pivot
	temp_df = add_prev_quarter_metrics(temp_df, quarter_col='week', prefix='week') #Thêm 2 cột 'week_prev_high', 'week_prev_low'
	temp_df = add_prev_quarter_metrics(temp_df, quarter_col='month', prefix='month') #Thêm 2 cột 'month_prev_high', 'month_prev_low'
	temp_df = add_prev_quarter_metrics(temp_df, quarter_col='quarter', prefix='quarter') #Thêm 2 cột 'quarter_prev_high', 'quarter_prev_low'
	temp_df = add_prev_quarter_metrics(temp_df, quarter_col='year', prefix='year') #Thêm 2 cột 'year_prev_high', 'year_prev_low'
      
	temp_df['WPIBOT_P'] = (temp_df['week_prev_high'] + temp_df['week_prev_low'] + temp_df['week_prev_close'])/3
	temp_df['WPIVOT_R1'] = 2*temp_df['WPIBOT_P'] - temp_df['week_prev_low']
	temp_df['WPIVOT_S1'] = 2*temp_df['WPIBOT_P'] - temp_df['week_prev_high']
	temp_df['MPIBOT_P'] = (temp_df['month_prev_high'] + temp_df['month_prev_low'] + temp_df['month_prev_close'])/3
	temp_df['mPIVOT_R1'] = 2*temp_df['MPIBOT_P'] - temp_df['month_prev_low']
	temp_df['mPIVOT_S1'] = 2*temp_df['MPIBOT_P'] - temp_df['month_prev_high']
	temp_df['QPIBOT_P'] = (temp_df['quarter_prev_high'] + temp_df['quarter_prev_low'] + temp_df['quarter_prev_close'])/3
	temp_df['QPIVOT_R1'] = 2*temp_df['QPIBOT_P'] - temp_df['quarter_prev_low']
	temp_df['QPIVOT_S1'] = 2*temp_df['QPIBOT_P'] - temp_df['quarter_prev_high']
	temp_df['YPIBOT_P'] = (temp_df['year_prev_high'] + temp_df['year_prev_low'] + temp_df['year_prev_close'])/3
	temp_df['YPIVOT_R1'] = 2*temp_df['YPIBOT_P'] - temp_df['year_prev_low']
	temp_df['YPIVOT_S1'] = 2*temp_df['YPIBOT_P'] - temp_df['year_prev_high']

	#Tính toán cho các chỉ báo khác vẽ cùng biểu đồ giá
	temp_df.ta.sma(close='close', length=5, append=True)  # Tạo cột 'SMA_5'
	temp_df.ta.sma(close='close', length=20, append=True)  # Tạo cột 'SMA_20'
	temp_df.ta.sma(close='close', length=60, append=True)  # Tạo cột 'SMA_60'
	temp_df.ta.sma(close='close', length=120, append=True)  # Tạo cột 'SMA_120'
	temp_df.ta.sma(close='close', length=240, append=True)  # Tạo cột 'SMA_240'
	temp_df.ta.sma(close='close', length=480, append=True)  # Tạo cột 'SMA_480'

	#Tính toán cho các chỉ báo vẽ dưới biểu đồ giá
	temp_df.ta.rsi(close='close', length=14, scalar=100, append=True)  # Tạo cột 'RSI_14'
	temp_df.ta.macd(close='close', fast=12, slow=26, signal=9, append=True) #Tạo cột 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9'
    
	history_stock_dict[ticker] = temp_df.sort_values('date', ascending=False).reset_index(drop=True)

history_stock_df = pd.DataFrame()
for ticker, df in history_stock_dict.items():
    temp_df = df[df['date'] != today].reset_index(drop=True)
    history_stock_df = pd.concat([history_stock_df, temp_df], axis=0)

##### Bảng lịch sử các nhóm cổ phiếu

- Lịch sử dữ liệu dòng tiền các nhóm cổ phiếu cho website

In [38]:
history_group_dict = {}
for ticker in group_stock_key_list:
    temp_df = date_series.sort_values('date')
    temp_df['ticker'] = ticker
    temp_df['close'] = group_price_index_df[ticker]
    temp_df['volume'] = group_volume_df[ticker]
    temp_df['vol_ratio'] = group_vol_ratio_df[ticker]
    temp_df['t0_score'] = group_score_df[ticker]
    temp_df['t5_score'] = temp_df['t0_score'][::-1].rolling(window=5, min_periods=1).mean()[::-1]
    temp_df['rank'] = group_score_ranking_df[ticker]

    history_group_dict[ticker] = temp_df.sort_values('date', ascending=False).reset_index(drop=True)

history_group_df = pd.DataFrame()
for ticker, df in history_group_dict.items():
    temp_df = df[df['date'] != today].reset_index(drop=True)
    history_group_df = pd.concat([history_group_df, temp_df], axis=0)

- Lịch sử dữ liệu âm dương dùng cho tín hiệu

In [39]:
group_score_positive_df['type'] = 'pos'
group_score_negative_df['type'] = 'neg'
history_portfolio_score_df = pd.concat([group_score_positive_df[group_score_positive_df['date'] != today], 
                                    group_score_negative_df[group_score_negative_df['date'] != today]], axis=0)

##### Bảng lịch sử các index

In [40]:
history_index_dict = {}
for ticker, df in index_dict.items():
	temp_df = df.sort_values('date').reset_index(drop=True)
      
	#Tạo thêm các cột mốc tuần tháng
	temp_df['week'] = temp_df['date'].dt.strftime('%Y-%W').apply(lambda x: f'{int(x[:4])-1}-53' if x[-2:] == '00' else x)
	temp_df['month'] = temp_df['date'].dt.strftime('%Y-%m')
	temp_df['quarter'] = temp_df['date'].apply(lambda x: next((key for key, value in quarter_timestamp_map_dict.items() if value[0] <= x <= value[1]), None))
	temp_df['year'] = temp_df['date'].dt.strftime('%Y')

	#Tính toán giá của các nến
	temp_df['week_high'] = temp_df.groupby('week')['high'].cummax()
	temp_df['week_low'] = temp_df.groupby('week')['low'].cummin()
	temp_df['week_open'] = temp_df.groupby('week')['open'].transform('first')     
	temp_df['month_high'] = temp_df.groupby('month')['high'].cummax()
	temp_df['month_low'] = temp_df.groupby('month')['low'].cummin()
	temp_df['month_open'] = temp_df.groupby('month')['open'].transform('first')
	temp_df['quarter_high'] = temp_df.groupby('quarter')['high'].cummax()
	temp_df['quarter_low'] = temp_df.groupby('quarter')['low'].cummin()
	temp_df['quarter_open'] = temp_df.groupby('quarter')['open'].transform('first')
	temp_df['year_high'] = temp_df.groupby('year')['high'].cummax()
	temp_df['year_low'] = temp_df.groupby('year')['low'].cummin()
	temp_df['year_open'] = temp_df.groupby('year')['open'].transform('first')

	#Tính toán cho chỉ báo Fibbonachi
	temp_df['2week_high'] = calculate_extreme_2period(temp_df, 'week', 'high')
	temp_df['2week_low'] = calculate_extreme_2period(temp_df, 'week', 'low')
	temp_df['2month_high'] = calculate_extreme_2period(temp_df, 'month', 'high')
	temp_df['2month_low'] = calculate_extreme_2period(temp_df, 'month', 'low')
	temp_df['2quarter_high'] = calculate_extreme_2period(temp_df, 'quarter', 'high')
	temp_df['2quarter_low'] = calculate_extreme_2period(temp_df, 'quarter', 'low')
	temp_df['2year_high'] = calculate_extreme_2period(temp_df, 'year', 'high')
	temp_df['2year_low'] = calculate_extreme_2period(temp_df, 'year', 'low')
			
	temp_df['WFIBO_0382'] = temp_df['2week_high'] - (temp_df['2week_high']-temp_df['2week_low'])*0.382
	temp_df['WFIBO_0500'] = temp_df['2week_high'] - (temp_df['2week_high']-temp_df['2week_low'])*0.500
	temp_df['WFIBO_0618'] = temp_df['2week_high'] - (temp_df['2week_high']-temp_df['2week_low'])*0.618
	temp_df['MFIBO_0382'] = temp_df['2month_high'] - (temp_df['2month_high']-temp_df['2month_low'])*0.382
	temp_df['MFIBO_0500'] = temp_df['2month_high'] - (temp_df['2month_high']-temp_df['2month_low'])*0.500
	temp_df['MFIBO_0618'] = temp_df['2month_high'] - (temp_df['2month_high']-temp_df['2month_low'])*0.618
	temp_df['QFIBO_0382'] = temp_df['2quarter_high'] - (temp_df['2quarter_high']-temp_df['2quarter_low'])*0.382
	temp_df['QFIBO_0500'] = temp_df['2quarter_high'] - (temp_df['2quarter_high']-temp_df['2quarter_low'])*0.500
	temp_df['QFIBO_0618'] = temp_df['2quarter_high'] - (temp_df['2quarter_high']-temp_df['2quarter_low'])*0.618
	temp_df['YFIBO_0382'] = temp_df['2year_high'] - (temp_df['2year_high']-temp_df['2year_low'])*0.382
	temp_df['YFIBO_0500'] = temp_df['2year_high'] - (temp_df['2year_high']-temp_df['2year_low'])*0.500
	temp_df['YFIBO_0618'] = temp_df['2year_high'] - (temp_df['2year_high']-temp_df['2year_low'])*0.618

	#Tính toán cho chỉ báo Pivot
	temp_df = add_prev_quarter_metrics(temp_df, quarter_col='week', prefix='week') #Thêm 2 cột 'week_prev_high', 'week_prev_low'
	temp_df = add_prev_quarter_metrics(temp_df, quarter_col='month', prefix='month') #Thêm 2 cột 'month_prev_high', 'month_prev_low'
	temp_df = add_prev_quarter_metrics(temp_df, quarter_col='quarter', prefix='quarter') #Thêm 2 cột 'quarter_prev_high', 'quarter_prev_low'
	temp_df = add_prev_quarter_metrics(temp_df, quarter_col='year', prefix='year') #Thêm 2 cột 'year_prev_high', 'year_prev_low'
			
	temp_df['WPIBOT_P'] = (temp_df['week_prev_high'] + temp_df['week_prev_low'] + temp_df['week_prev_close'])/3
	temp_df['WPIVOT_R1'] = 2*temp_df['WPIBOT_P'] - temp_df['week_prev_low']
	temp_df['WPIVOT_S1'] = 2*temp_df['WPIBOT_P'] - temp_df['week_prev_high']
	temp_df['MPIBOT_P'] = (temp_df['month_prev_high'] + temp_df['month_prev_low'] + temp_df['month_prev_close'])/3
	temp_df['mPIVOT_R1'] = 2*temp_df['MPIBOT_P'] - temp_df['month_prev_low']
	temp_df['mPIVOT_S1'] = 2*temp_df['MPIBOT_P'] - temp_df['month_prev_high']
	temp_df['QPIBOT_P'] = (temp_df['quarter_prev_high'] + temp_df['quarter_prev_low'] + temp_df['quarter_prev_close'])/3
	temp_df['QPIVOT_R1'] = 2*temp_df['QPIBOT_P'] - temp_df['quarter_prev_low']
	temp_df['QPIVOT_S1'] = 2*temp_df['QPIBOT_P'] - temp_df['quarter_prev_high']
	temp_df['YPIBOT_P'] = (temp_df['year_prev_high'] + temp_df['year_prev_low'] + temp_df['year_prev_close'])/3
	temp_df['YPIVOT_R1'] = 2*temp_df['YPIBOT_P'] - temp_df['year_prev_low']
	temp_df['YPIVOT_S1'] = 2*temp_df['YPIBOT_P'] - temp_df['year_prev_high']

	#Tính toán cho các chỉ báo khác vẽ cùng biểu đồ giá
	temp_df.ta.sma(close='close', length=5, append=True)  # Tạo cột 'SMA_5'
	temp_df.ta.sma(close='close', length=20, append=True)  # Tạo cột 'SMA_20'
	temp_df.ta.sma(close='close', length=60, append=True)  # Tạo cột 'SMA_60'
	temp_df.ta.sma(close='close', length=120, append=True)  # Tạo cột 'SMA_120'
	temp_df.ta.sma(close='close', length=240, append=True)  # Tạo cột 'SMA_240'
	temp_df.ta.sma(close='close', length=480, append=True)  # Tạo cột 'SMA_480'

	#Tính toán cho các chỉ báo vẽ dưới biểu đồ giá
	temp_df.ta.rsi(close='close', length=14, scalar=100, append=True)  # Tạo cột 'RSI_14'
	temp_df.ta.macd(close='close', fast=12, slow=26, signal=9, append=True) #Tạo cột 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9'

	history_index_dict[ticker] = temp_df.sort_values('date', ascending=False).reset_index(drop=True)

history_index_df = pd.DataFrame()
for ticker, df in history_index_dict.items():
    temp_df = df[df['date'] != today].reset_index(drop=True)
    history_index_df = pd.concat([history_index_df, temp_df], axis=0)

#### Lưu dữ liệu history vào mongo

In [41]:
overwrite_mongo(ref_db["current_quarter_classification"], quarter_stock_classification_dict[get_quarter('current_quarter')])
overwrite_mongo(ref_db["current_total_stock_list"], pd.DataFrame(total_stock_list).rename(columns={0:'ticker'}))
overwrite_mongo(ref_db["date_series"], date_series)
        
overwrite_mongo(stock_db["history_stock"], history_stock_df)
overwrite_mongo(stock_db["history_index"], history_index_df)
overwrite_mongo(stock_db["history_ms_chart"], history_ms_chart_df)
overwrite_mongo(stock_db["history_group"], history_group_df)
overwrite_mongo(stock_db["history_portfolio_score"], history_portfolio_score_df)

### Phần dữ liệu lịch sử danh mục

#### Định nghĩa các biến porfolio

- Tạo các biến portfolio để tính toán danh mục

In [42]:
# Dữ liệu cổ phiếu full
portfolio_stock_dict = {}
for ticker, df in history_stock_dict.items():
    portfolio_stock_dict[ticker] = df[["ticker", "date", "open", "high", "low", "close", 'volume', 'vol_ratio', "t0_score", "t5_score", "WFIBO_0618", "MFIBO_0618", "WFIBO_0382"]]

# Dữ liệu vnindex full
portfolio_index_df = history_index_dict['VNINDEX'].reset_index(drop=True)

# Dữ liệu dòng tiền nhóm cổ phiếu full
portfolio_portfolio_score_df = pd.concat([group_score_positive_df, group_score_negative_df], axis=0)

#Đọc dữ liệu ms lịch sử
portfolio_ms_chart_df = all_stock_ms['all'].sort_values('date', ascending=False).reset_index(drop=True)

- Chỉnh sửa lại portfolio_stock_dict để thêm dữ liệu

In [43]:
def round_price_by_step(df, value_column, step_size_column):
    # Tạo một bản sao để tránh thay đổi df gốc
    result = df[value_column].copy()
    
    # Trường hợp step_size > 0: làm tròn theo bội số
    mask = df[step_size_column] > 0
    if mask.any():
        result.loc[mask] = (df.loc[mask, value_column] / df.loc[mask, step_size_column]).round() * df.loc[mask, step_size_column]
    
    return result

In [44]:
#Tạo lại portfolio_stock_dict từ dữ liệu monggoDB
for ticker, df in portfolio_stock_dict.items():
    temp_df = df.copy()
    temp_df['t0_score'] = temp_df['t0_score'].fillna(0)
    temp_df['t5_score'] = temp_df['t5_score'].fillna(0)
    temp_df['exchange'] = stock_classification_df.set_index('ticker').at[ticker, 'exchange']
    temp_df['price_step'] = temp_df.apply(lambda x: 0.01 if (x['exchange'] == 'HSX') & (x['close'] < 10) else (0.05 if (x['exchange'] == 'HSX') & (x['close'] < 50) else 0.1), axis=1)
    temp_df['pct_change'] = temp_df['close'][::-1].pct_change()[::-1].fillna(0)

    #Các cột biến động giá cần dùng
    temp_df['highest_2'] = temp_df['high'][::-1].rolling(window=2, min_periods=1).max()[::-1]
    temp_df['diff_highest2'] = (temp_df['close'] - temp_df['highest_2'])/temp_df['highest_2']
    temp_df['lowest_10'] = temp_df['low'][::-1].rolling(window=10, min_periods=1).min()[::-1]
    temp_df['diff_lowest10'] = (temp_df['close'] - temp_df['lowest_10'])/temp_df['lowest_10']
    temp_df['highest_10'] = temp_df['high'][::-1].rolling(window=10, min_periods=1).max()[::-1]
    temp_df['diff_highest10'] = (temp_df['close'] - temp_df['highest_10'])/temp_df['highest_10']
    
    #Các cột khối lượng giao dịch để lọc
    temp_df['min_vol'] = temp_df['volume'][::-1].rolling(window=10, min_periods=1).min()[::-1]
    temp_df['mean_vol'] = temp_df['volume'][::-1].rolling(window=60, min_periods=1).mean()[::-1]

    #Các cột giá trị giao dịch để lọc
    temp_df['value_traded'] = temp_df['close']*temp_df['volume']
    temp_df['min_value'] = temp_df['value_traded'][::-1].rolling(window=10, min_periods=1).min()[::-1]
    temp_df['mean_value'] = temp_df['value_traded'][::-1].rolling(window=60, min_periods=1).mean()[::-1]

    #Các cột tính các mốc giá mua bán khuyến nghị trong phiên
    temp_df['calculate_high'] = np.maximum(temp_df['high'], temp_df['close'].shift(-1).ffill()*1.015)
    temp_df['calculate_low'] = np.minimum(temp_df['low'], temp_df['close'].shift(-1).ffill()*0.985)

    temp_df['trade_high'] = temp_df['calculate_high'] - (temp_df['calculate_high'] - np.minimum(temp_df['open'], temp_df['close']))*0.382
    temp_df['trade_mean'] = (temp_df['close'] + temp_df['open'])/2
    temp_df['trade_low'] = temp_df['calculate_low'] + (np.maximum(temp_df['open'], temp_df['close']) - temp_df['calculate_low'])*0.382

    temp_df['trade_high'] = round_price_by_step(temp_df, 'trade_high', 'price_step')
    temp_df['trade_mean'] = round_price_by_step(temp_df, 'trade_mean', 'price_step')
    temp_df['trade_low'] = round_price_by_step(temp_df, 'trade_low', 'price_step')

    # Các giá trị cơ sở khi không có biến động giá
    base_high = 0.30
    base_mean = 0.40
    base_low = 0.30
    
    # Hệ số điều chỉnh với tỉ lệ không đối xứng
    high_factor = 4.0  # Hệ số tăng mạnh cho portion_high khi giá tăng
    low_factor = 4.0   # Hệ số tăng mạnh cho portion_low khi giá giảm
    mean_factor = 2.0  # Hệ số điều chỉnh nhẹ cho portion_mean
    
    # Khởi tạo các cột portion mặc định
    temp_df['portion_high'] = base_high
    temp_df['portion_mean'] = base_mean
    temp_df['portion_low'] = base_low
    
    # Xử lý khi giá tăng (pct_change > 0)
    increase_mask = temp_df['pct_change'] > 0
    if increase_mask.any():
        # portion_high tăng nhiều
        temp_df.loc[increase_mask, 'portion_high'] = base_high + temp_df.loc[increase_mask, 'pct_change'] * high_factor
        # portion_mean giảm nhẹ
        temp_df.loc[increase_mask, 'portion_mean'] = base_mean - temp_df.loc[increase_mask, 'pct_change'] * mean_factor * 0.5
        # portion_low giảm ít
        temp_df.loc[increase_mask, 'portion_low'] = 1 - temp_df.loc[increase_mask, 'portion_high'] - temp_df.loc[increase_mask, 'portion_mean']

    # Xử lý khi giá giảm (pct_change < 0)
    decrease_mask = temp_df['pct_change'] < 0
    if decrease_mask.any():
        # portion_low tăng nhiều 
        temp_df.loc[decrease_mask, 'portion_low'] = base_low - temp_df.loc[decrease_mask, 'pct_change'] * low_factor
        # portion_mean giảm nhẹ
        temp_df.loc[decrease_mask, 'portion_mean'] = base_mean + temp_df.loc[decrease_mask, 'pct_change'] * mean_factor * 0.5
        # portion_high giảm ít
        temp_df.loc[decrease_mask, 'portion_high'] = 1 - temp_df.loc[decrease_mask, 'portion_low'] - temp_df.loc[decrease_mask, 'portion_mean']
    
    # Đảm bảo tổng tỉ trọng = 1
    temp_df['portion_mean'] = 1 - temp_df['portion_high'] - temp_df['portion_low']

    #Tính giá cuối cùng để tính hiệu suất mua bán
    temp_df['trade_price'] = temp_df['portion_high']*temp_df['trade_high'] + temp_df['portion_mean']*temp_df['trade_mean'] + temp_df['portion_low']*temp_df['trade_low']
    temp_df['trade_price'] = round_price_by_step(temp_df, 'trade_price', 'price_step')
    
    portfolio_stock_dict[ticker] = temp_df

#### Tín hiệu toàn thị trường

##### Hàm tạo bảng markert_phase

In [45]:
def calculate_market_ms(ms_df):
    # Tạo bản sao của DataFrame để không làm thay đổi dữ liệu gốc
    df = ms_df.copy()

    # --- Tính toán các cột shift cho trend_5p ---
    df['5p_shift1'] = df['trend_5p'].shift(-1)
    df['5p_shift2'] = df['trend_5p'].shift(-2)
    df['5p_shift5'] = df['trend_5p'][::-1].rolling(window=5, min_periods=1).mean()[::-1].shift(-1)
    df['5p_diff1'] = df['trend_5p'][::-1].diff()[::-1]
    df['5p_diff5'] = df['trend_5p'] - df['5p_shift5']
    
    # --- Tính toán các cột shift cho trend_20p ---
    df['20p_shift1'] = df['trend_20p'].shift(-1)
    df['20p_shift2'] = df['trend_20p'].shift(-2)
    df['20p_shift5'] = df['trend_20p'][::-1].rolling(window=5, min_periods=1).mean()[::-1].shift(-1)
    df['20p_diff1'] = df['trend_20p'][::-1].diff()[::-1]
    df['20p_diff5'] = df['trend_20p'] - df['20p_shift5']

    # --- Tính toán các cột shift cho trend_60p ---
    df['60p_shift1'] = df['trend_60p'].shift(-1)
    df['60p_shift2'] = df['trend_60p'].shift(-2)

    # --- Tín hiệu mua (buy) ---
    # So sánh giá trị hiện tại với các giá trị dịch chuyển
    df['buy_ms5p_check'] = ((df['trend_5p'] > df['5p_shift1']) & (df['trend_5p'] > df['5p_shift5'])).astype(int)
    df['buy_ms20p_check'] = ((df['trend_20p'] > df['20p_shift1']) & (df['trend_20p'] > df['20p_shift5'])).astype(int)
    df['buy_ms60p_check'] = ((df['trend_60p'] > df['60p_shift1']) & (df['trend_60p'] > df['60p_shift2'])).astype(int)

    # Kiểm tra xem giá có ở mức quá cao không
    df['buy_overprice_check'] = ((df['5p_diff1'] > 0.4) | ((df['trend_5p'] < 0.9) & (df['trend_20p'] < 0.9))).astype(int)

    # Kiểm tra giá trị biến động của xu hướng
    buy_threshold_5p = 0.3
    buy_threshold_20p = 0.2
    df['buy_msvalue_check'] = ( (df['5p_diff1'] > 0.41) | 
                               
            (((df['5p_shift1'] > buy_threshold_5p) & (df['trend_5p'] > buy_threshold_5p)) | ((df['5p_shift1'] < buy_threshold_5p) & (df['trend_5p'] > buy_threshold_5p) & (df['5p_diff1']*0.3 < (df['trend_5p'] - buy_threshold_5p)))) &
            (((df['20p_shift1'] > buy_threshold_20p) & (df['trend_20p'] > buy_threshold_20p)) | ((df['20p_shift1'] < buy_threshold_20p) & (df['trend_20p'] > buy_threshold_20p) & (df['20p_diff1']*0.3 < (df['trend_20p'] - buy_threshold_20p))))
        ).astype(int)

    # Tính tín hiệu mua dựa trên tất cả các điều kiện
    df['buy_ms_check'] = df.apply(
        lambda x: 1 if (x['buy_ms5p_check'] == 1 and x['buy_ms20p_check'] == 1 and 
                        x['buy_ms60p_check'] == 1 and x['buy_overprice_check'] == 1 and 
                        x['buy_msvalue_check'] == 1)
                  else (2 if x['buy_overprice_check'] == 0 else 0),
        axis=1
    )

    # --- Tín hiệu bán (sell) ---
    df['sell_ms5p_check'] = ((df['trend_5p'] < df['5p_shift1']) | (df['trend_5p'] < df['5p_shift2'])).astype(int)
    df['sell_ms20p_check'] = ((df['trend_20p'] < df['20p_shift1']) | (df['trend_20p'] < df['20p_shift2'])).astype(int)
    df['sell_ms60p_check'] = ((df['trend_60p'] < df['60p_shift1']) & (df['trend_60p'] < df['60p_shift2'])).astype(int)

    # Kiểm tra xem giá có ở mức quá thấp không (underprice)
    df['sell_underprice_check'] = ((df['trend_5p'] > 0.22) | (df['trend_20p'] > 0.25)).astype(int)

    # Kiểm tra giá trị biến động cho tín hiệu bán
    sell_threshold_5p = 0.4
    sell_threshold_20p = 0.5
    df['sell_msvalue_check'] = ( (df['5p_diff1'] < -0.41) |
                                
        (((df['5p_shift1'] < sell_threshold_5p) & (df['trend_5p'] < sell_threshold_5p)) | ((df['5p_shift1'] > sell_threshold_5p) & (df['trend_5p'] < sell_threshold_5p) & (-df['5p_diff1']*0.3 < (sell_threshold_5p - df['trend_5p'])))) &
        (((df['20p_shift1'] < sell_threshold_20p) & (df['trend_20p'] < sell_threshold_20p)) | ((df['20p_shift1'] > sell_threshold_20p) & (df['trend_20p'] < sell_threshold_20p) & (-df['20p_diff1']*0.3 < (sell_threshold_20p - df['trend_20p'])))) &
        
        (df['trend_60p'] < 0.7)
    ).astype(int)
    
    # Tính tín hiệu bán dựa trên tất cả các điều kiện
    df['sell_ms_check'] = df.apply(
        lambda x: 1 if (x['sell_ms5p_check'] == 1 and x['sell_ms20p_check'] == 1 and 
                        x['sell_underprice_check'] == 1 and x['sell_msvalue_check'] == 1)
                  else (2 if x['sell_underprice_check'] == 0 else 0),
        axis=1
    )

    return df[['date','trend_5p','trend_20p','trend_60p','buy_ms5p_check','buy_ms20p_check','buy_ms60p_check','buy_overprice_check','buy_msvalue_check','sell_ms5p_check','sell_ms20p_check','sell_ms60p_check','sell_underprice_check','sell_msvalue_check','buy_ms_check','sell_ms_check']]

In [46]:
def calculate_market_phase_raw(df):
    df_copy = df.copy()

    conditions = [
        df_copy['buy_ms_check'] == 2,
        (df_copy['buy_ms_check'] == 1) & (df_copy['buy_score_check'] == 1),
        df_copy['sell_ms_check'] == 1,
        df_copy['sell_ms_check'] == 2
    ]
    choices = [2, 1, -1, -2]
    
    # Tạo cột market_phase_raw theo điều kiện, nếu không thỏa thì gán NaN
    df_copy['market_phase_raw'] = np.select(conditions, choices, default=np.nan)
    
    # Sử dụng backfill để lấp đầy các giá trị NaN
    df_copy['market_phase_raw'] = df_copy['market_phase_raw'].bfill()
    
    # Đảm bảo dòng cuối cùng luôn là 1 (mua)
    df_copy.loc[df_copy.index[-1], 'market_phase_raw'] = 1
    
    return df_copy['market_phase_raw']

In [47]:
def adjust_market_phase(df):
    """
    Hàm nhận vào DataFrame có cột 'market_phase_raw' (các giá trị: 1, 2, -1, -2)
    và tạo cột binary (1: mua, 0: bán) theo quy tắc kiểm tra từ dưới lên trên với “khóa” trạng thái:
    
      - Dòng cuối (thời gian mới nhất) được xác định theo dấu của market_phase_raw:
            Nếu > 0 -> 1 (mua), nếu < 0 -> 0 (bán).
      - Khi duyệt ngược từ dưới lên trên:
          + Nếu trạng thái hiện hành là mua (state > 0):
                - Nếu gặp tín hiệu âm:
                      * Nếu tín hiệu là -1: chuyển sang bán (state = -1).
                      * Nếu tín hiệu là -2: giữ mua (không cập nhật state).
                - Nếu gặp tín hiệu dương:
                      * Nếu state hiện hành là 2 và tín hiệu mới là 1, thì giữ lại state = 2.
                      * Các trường hợp khác: cập nhật state = tín hiệu mới.
          + Nếu trạng thái hiện hành là bán (state < 0):
                - Nếu gặp tín hiệu dương:
                      * Nếu tín hiệu là 1: chuyển sang mua (state = 1).
                      * Nếu tín hiệu là 2: giữ bán (không cập nhật state).
                - Nếu gặp tín hiệu âm:
                      * Nếu state hiện hành là -2 và tín hiệu mới là -1, thì giữ lại state = -2.
                      * Các trường hợp khác: cập nhật state = tín hiệu mới.
          + Sau mỗi bước, gán binary[i] = 1 nếu state > 0, ngược lại 0.
    """
    phases = df['market_phase_raw'].tolist()
    n = len(phases)
    binary = [None] * n

    # Dòng cuối cùng: xác định trạng thái theo tín hiệu của nó
    state = phases[-1]
    binary[-1] = 1 if state > 0 else 0

    # Duyệt ngược từ dòng áp chót về đầu DataFrame
    for i in range(n - 2, -1, -1):
        p = phases[i]
        if state > 0:  # Hiện đang ở trạng thái mua
            if p < 0:
                # Nếu tín hiệu âm: chỉ chuyển nếu là -1 (đảo chiều)
                if p == -1:
                    state = -1
                elif p == -2:
                    # Tín hiệu -2 không làm chuyển trạng thái, giữ mua
                    pass
            elif p > 0:
                # Nếu tín hiệu dương: kiểm tra "khóa" trạng thái mua
                # Nếu đang ở phase 2 và tín hiệu mới là 1, thì giữ phase 2 (không giảm xuống phase 1)
                if state == 2 and p == 1:
                    pass
                else:
                    state = p
            # Nếu p == 0 (không có tín hiệu) thì giữ nguyên state
        else:  # state < 0, hiện đang ở trạng thái bán
            if p > 0:
                # Nếu tín hiệu dương: chỉ chuyển nếu tín hiệu là 1 (đảo chiều)
                if p == 1:
                    state = 1
                elif p == 2:
                    # Tín hiệu 2 không làm chuyển trạng thái, giữ bán
                    pass
            elif p < 0:
                # Nếu tín hiệu âm: kiểm tra "khóa" trạng thái bán
                # Nếu đang ở phase -2 và tín hiệu mới là -1, thì giữ phase -2 (không giảm xuống -1)
                if state == -2 and p == -1:
                    pass
                else:
                    state = p
            # Nếu p == 0 thì giữ nguyên
        binary[i] = 1 if state > 0 else 0

    return binary


In [48]:
def final_market_phase_T3(df):
    # Tạo bản sao của DataFrame để không làm thay đổi dữ liệu gốc
    df_copy = df.copy()
    
    # Tạo cột market_phase_final dựa trên cột market_phase gốc
    df_copy['market_phase_final'] = df_copy['market_phase_adj']
    # Tạo cột tín hiệu T3, khởi tạo giá trị bằng 0
    df_copy['t3_signal'] = 0

    # Lặp ngược từ len(df_copy)-4 đến 0 để đảm bảo các phiên sau đã được xử lý
    for i in range(len(df_copy) - 4, -1, -1):
        current = df_copy['market_phase_final'].iloc[i]
        next_val = df_copy['market_phase_final'].iloc[i+1]
        next2 = df_copy['market_phase_final'].iloc[i+2]
        next3 = df_copy['market_phase_final'].iloc[i+3]

        # Nếu giá trị của phiên tiếp theo lớn hơn phiên sau và lớn hơn phiên hiện tại
        if (next_val > next2) and (next_val > current):
            # Cập nhật market_phase_final tại vị trí i với giá trị của phiên tiếp theo
            df_copy.at[df_copy.index[i], 'market_phase_final'] = next_val
            # Gán tín hiệu T3 bằng 2 (còn 2 phiên nữa mới bán được)
            df_copy.at[df_copy.index[i], 't3_signal'] = 2
        # Nếu giá trị của phiên thứ 2 sau lớn hơn phiên thứ 3 và phiên tiếp theo lớn hơn phiên hiện tại
        elif (next2 > next3) and (next_val > current):
            df_copy.at[df_copy.index[i], 'market_phase_final'] = next_val
            # Gán tín hiệu T3 bằng 1 (còn 1 phiên nữa mới bán được)
            df_copy.at[df_copy.index[i], 't3_signal'] = 1

    # Trả về 2 cột mới: market_phase_final và t3_signal
    return df_copy['market_phase_final']


##### Hàm lọc danh sách cổ phiếu từng quý

In [49]:
def calculate_capital(df, return_column, initial_capital):
    """
    Hàm tính số tiền sau mỗi ngày dựa trên cột return_column (pct return) và số vốn ban đầu.
    Quy tắc tính: 
        capital[t] = initial_capital * (1 + return[t1]) * (1 + return[t2]) * ... * (1 + return[t])
    Trong đó chuỗi tính theo thứ tự thời gian từ ngày cũ nhất đến ngày hiện tại.
    
    Vì DataFrame được sắp xếp giảm dần (hàng đầu tiên là ngày hiện tại),
    ta cần đảo ngược thứ tự để tính tích lũy, sau đó gán kết quả lại cho df ban đầu.
    
    Trả về Series 'capital'.
    """
    df_copy = df.iloc[::-1].copy()
    df_copy['capital'] = initial_capital * (1 + df_copy[return_column]).cumprod()
    return df_copy['capital'][::-1]

def calculate_pct_return(df, phase_column):
    df_copy = df.copy()
    df_copy['return'] = 0
    df_copy['phase_s1'] = df_copy[phase_column].shift(-1).ffill()
    df_copy['phase_s2'] = df_copy[phase_column].shift(-2).ffill()
    df_copy['close_s1'] = df_copy['close'].shift(-1).ffill()

    df_copy['return'] = df_copy.apply(lambda x: 
        #Ngày đầu tiên có tín hiệu, ví là tín hiệu vào cuối phiên nên ko tính return
        0 if (x[phase_column]==1) & (x['phase_s1']==0) & (x['phase_s2']==0) else (
        #Ngày đầu tiên tính return, tức là ngày thứ 2 kể từ cuối phiên biết có tín hiệu
        (x['close'] - x['open'])/x['open'] if (x[phase_column]==1) & (x['phase_s1']==1) & (x['phase_s2']==0) else (
        #Các ngày giữa chu kì mua thì tính return bằng giá close như bình thường
        (x['close'] - x['close_s1'])/x['close_s1'] if (x[phase_column]==1) & (x['phase_s1']==1) & (x['phase_s2']==1) else (
        #Ngày đầu tiên có tín hiệu bán, nhưng chưa thể bán ngay nên vẫn tính như thường
        (x['close'] - x['close_s1'])/x['close_s1'] if (x[phase_column]==0) & (x['phase_s1']==1) & (x['phase_s2']==1) else (
        #Sau khi có tín hiệu bán 1 ngày, tính return dựa vào giá open vì sẽ bán ATO
        (x['open'] - x['close_s1'])/x['close_s1'] if (x[phase_column]==0) & (x['phase_s1']==0) & (x['phase_s2']==1) else 
        #Các ngày khác return lại tính bằng 0
        0)))), axis=1)
    
    return df_copy['return']

#Hàm tính toán hiệu suất sử dụng cho từng chổ phiếu
def calculate_ticker_return(df):
    ticker_perform_df = df.copy()
    ticker_perform_df['ticker_phase_final'] = market_phase_df['market_phase_final']
    ticker_perform_df['ticker_phase_shifted'] = ticker_perform_df['ticker_phase_final'].shift(-1).ffill()
    ticker_perform_df['pct_change'] = ticker_perform_df['close'][::-1].pct_change()[::-1]
    ticker_perform_df['ticker_return'] = ticker_perform_df['pct_change']*ticker_perform_df['ticker_phase_shifted']

    return ticker_perform_df['ticker_return']

#Hàm cắt stock dict ra thành từng quý, sau đó tính toán performance của từng quý để lọc cổ phiếu có performance thấp
def create_quarter_stock_dict_for_signal(portfolio_stock_dict, raw_signal_stock_list, quarter_map_dict, quarter_stock_map):

    quarter_series = pd.Series(quarter_map_dict.keys())

    #Tạo danh sách cổ phiếu cho từng quý
    signal_quarter_stock_df_dict = {}
    temp_stock_dict = {}
    for ticker in raw_signal_stock_list:
        temp_df = portfolio_stock_dict[ticker].copy()
        temp_df['ticker_return'] = calculate_ticker_return(temp_df)
        temp_stock_dict[ticker] = temp_df
        
    for i in range(len(quarter_series)):
        quarter = quarter_series[i] # Key của quarter map dict
        
        if i == 0:
            time_range = quarter_map_dict[quarter_series[i]] # Value của quarter map dict
        else:
            time_range = quarter_map_dict[quarter_series[i-1]] # Value của quarter map dict

        quarter_df = pd.DataFrame()
        for ticker, df in temp_stock_dict.items():
            temp_df = df[['date', 'ticker', 'open', 'high', 'low', 'close', 'volume', 'ticker_return']]
            temp_df = temp_df[(temp_df['date'] >= time_range[0]) & (temp_df['date'] <= time_range[1])]
            temp_df['cum_return'] = temp_df['ticker_return'][::-1].cumsum()[::-1]
            temp_df = temp_df.sort_values('date', ascending=False).iloc[[0]]
            quarter_df = pd.concat([quarter_df, temp_df], axis=0)
        
        quarter_df = quarter_df[quarter_df['ticker'].isin(quarter_stock_map[quarter].dropna().tolist())]
        signal_quarter_stock_df_dict[quarter] = quarter_df


    return signal_quarter_stock_df_dict

#Hàm lọc cổ phiếu theo quý dựa trên hiệu suất với quý trước và gộp vào cùng danh sách cổ phiếu bắt buộc
def filter_quarter_stock_list_dict(signal_quarter_stock_df_dict, mandatory_stock_list, prohibited_stock_list):
    temp_quarter_stock_list_dict = {}
    # Convert prohibited list to a set once for efficiency
    prohibited_set = set(prohibited_stock_list)

    for quarter in quarter_name_list:
        temp_df = signal_quarter_stock_df_dict[quarter].copy()
        # temp_df = temp_df.sort_values('cum_return', ascending=False).head(int(len(temp_df)/2))

        mean_return = temp_df['cum_return'].mean()
        temp_df = temp_df[temp_df['cum_return'] > 1.5*mean_return]
        
        # Step 1: Get stocks passing performance filter as a set
        performance_stocks = set(temp_df['ticker'].tolist())
        
        # Step 2: Add mandatory stocks
        candidate_stocks = performance_stocks.union(mandatory_stock_list)
        
        # Step 4: Final filtering - keep stocks that are candidates AND available BUT NOT prohibited
        final_stocks = candidate_stocks.difference(prohibited_set)
        
        # Convert to sorted list for consistent output
        temp_quarter_stock_list_dict[quarter] = sorted(list(final_stocks))

    return temp_quarter_stock_list_dict

#Hàm tính toán trung bình thay đổi của tất cả cổ phiếu trong danh sách
def calculate_total_change(stock_group, price_index_date_series):
    quarter_index_df = price_index_date_series.copy()

    for ticker, df in stock_group.items():
        quarter_index_df[ticker] = df['open']
        quarter_index_df[ticker] = quarter_index_df[ticker][::-1].pct_change()[::-1]

    quarter_index_df['total_change'] = quarter_index_df.iloc[:,1:].mean(axis=1)
    quarter_index_df = quarter_index_df.fillna(0)

    return quarter_index_df

##### Tạo bảng market_phase và danh sách cổ phiếu từng quý

- Tạo bảng market phase 

In [50]:
#Tách bảng điểm dòng tiền thanh bảng dòng tiền âm và dương
group_positive_df = portfolio_portfolio_score_df[portfolio_portfolio_score_df['type']=='pos'].drop(columns=['type']).reset_index(drop=True)
group_negative_df = portfolio_portfolio_score_df[portfolio_portfolio_score_df['type']=='neg'].drop(columns=['type']).reset_index(drop=True)

#Tính phân bổ vốn hoá tổng chưa điều chỉnh
market_phase_df = date_series.copy()

#Tính dòng tiền vào và ra trung trung trong 5 phiên
market_phase_df[['hsA+','hsB+','hsC+','hsD+']] = group_positive_df[industry_perform_list][::-1].rolling(window=5, min_periods=1).mean()[::-1]
market_phase_df[['hsA-','hsB-','hsC-','hsD-']] = group_negative_df[industry_perform_list][::-1].rolling(window=5, min_periods=1).mean().abs()[::-1]

#Tính tỉ lệ dòng tiền vào/ra, nhân theo tỉ lệ và tạo cột score check để kiểm tra điều kiện dòng tiền
market_phase_df['hsA_score'] = ((market_phase_df['hsA+'] - market_phase_df['hsA-']) / (market_phase_df['hsA+'] + market_phase_df['hsA-'])) * 0.3
market_phase_df['hsB_score'] = (market_phase_df['hsB+'] - market_phase_df['hsB-']) / (market_phase_df['hsB+'] + market_phase_df['hsB-']) * 0.3
market_phase_df['hsC_score'] = (market_phase_df['hsC+'] - market_phase_df['hsC-']) / (market_phase_df['hsC+'] + market_phase_df['hsC-']) * 0.3
market_phase_df['hsD_score'] = (market_phase_df['hsD+'] - market_phase_df['hsD-']) / (market_phase_df['hsD+'] + market_phase_df['hsD-']) * 0.1

#Kiểm tra dòng tiền hiệu suất A
market_phase_df['buy_hsA_score_check'] = (market_phase_df['hsA_score'] > 0.12).astype(int)
market_phase_df['sell_hsA_score_check'] = (market_phase_df['hsA_score'] <= 0.12).astype(int)

#Kiểm tra dòng tiền hiệu suất C
market_phase_df['hsC_score_diff1'] = market_phase_df['hsC_score'][::-1].diff()[::-1].fillna(0)
market_phase_df['buy_hsC_score_check'] = (market_phase_df['hsC_score_diff1'] > 0).astype(int)
market_phase_df['sell_hsC_score_check'] = (market_phase_df['hsC_score_diff1'] <= 0).astype(int)

#Kiểm tra dòng tiền thị trường
market_phase_df['market_score'] = market_phase_df[['hsA_score','hsB_score','hsC_score','hsD_score']].sum(axis=1)
market_phase_df['buy_market_score_check'] = (market_phase_df['market_score'] > 0.2).astype(int)
market_phase_df['sell_market_score_check'] = (market_phase_df['market_score'] <= 0.2).astype(int)

#Tín hiệu tổng hợp dòng tiền
market_phase_df['buy_score_check'] = market_phase_df.apply(lambda x: 1 if (x['buy_market_score_check'] == 1) & (x['buy_hsA_score_check'] == 1)  & (x['buy_hsC_score_check'] == 1) else 0, axis=1)
market_phase_df['sell_score_check'] = (market_phase_df['market_score'] > 0).astype(int)

market_phase_df = market_phase_df.drop(columns=['hsA+', 'hsB+', 'hsC+', 'hsD+', 'hsA-', 'hsB-', 'hsC-', 'hsD-', 'hsC_score_diff1'])

#Ghép bảng phase với bảng MS để thêm các cột tín hiệu từ MS
market_ms_chart_df = portfolio_ms_chart_df[['date','trend_5p','trend_20p','trend_60p']]
market_ms_chart_df = calculate_market_ms(market_ms_chart_df).sort_values('date', ascending=False).reset_index(drop=True)
market_phase_df = market_phase_df.merge(market_ms_chart_df, on='date', how='left')

#Điều chỉnh lại các tín hiệu dummy của sell cho đồng nhất tín hiệu tổng
market_phase_df['sell_hsA_score_check'] = market_phase_df.apply(lambda x: 1 if x['sell_ms_check'] == 1 else x['sell_hsA_score_check'], axis=1)
market_phase_df['sell_hsC_score_check'] = market_phase_df.apply(lambda x: 1 if x['sell_ms_check'] == 1 else x['sell_hsC_score_check'], axis=1)
market_phase_df['sell_market_score_check'] = market_phase_df.apply(lambda x: 1 if x['sell_ms_check'] == 1 else x['sell_market_score_check'], axis=1)
market_phase_df['sell_score_check'] = market_phase_df.apply(lambda x: 1 if x['sell_ms_check'] == 1 else x['sell_market_score_check'], axis=1)
market_phase_df['sell_ms60p_check'] = market_phase_df.apply(lambda x: 1 if x['sell_ms_check'] == 1 else x['sell_ms60p_check'], axis=1)

#Điều chỉnh lại các giai đoạn mua bán tạo thành 2 giai đoạn chính mua (1) và bán (0)
market_phase_df['market_phase_raw'] = calculate_market_phase_raw(market_phase_df)
market_phase_df['market_phase_adj'] = adjust_market_phase(market_phase_df)
market_phase_df['market_phase_final'] = final_market_phase_T3(market_phase_df)


- Tạo dict chứa danh sách cổ phiếu từng quý sau khi lọc

In [51]:
mandatory_stock_list = stock_classification_df[stock_classification_df['signal']=='o']['ticker'].tolist()
prohibited_stock_list  = stock_classification_df[stock_classification_df['signal']=='x']['ticker'].tolist()

#Hàm cắt stock dict ra thành từng quý, sau đó tính toán performance quý cho từng cổ phiếu
signal_quarter_stock_df_dict = create_quarter_stock_dict_for_signal(portfolio_stock_dict, total_stock_list, quarter_map_dict, quarter_stock_map)

#Tạo dict cổ phiếu sau khi lọc qua điều kiện performance và kết hợp với danh sách cổ phiếu bắt buộc
signal_quarter_stock_list_dict = filter_quarter_stock_list_dict(signal_quarter_stock_df_dict, mandatory_stock_list, prohibited_stock_list)

#### Bộ phân bổ vốn tự động

##### Các hàm tính toán

In [52]:
#Hàm này trả về các nhóm hsA hoặc hsBC để tính toán trong các hàm dưới
def calculate_group_industry_perform(industry_perform):
    if industry_perform == 'hsA':
        return 'hsA'
    elif (industry_perform == 'hsB') | (industry_perform == 'hsC') | (industry_perform == 'hsD'):
        return 'hsBCD'

#Hàm này tính toán tỉ trọng mỗi khi pick cổ phiếu
def calculate_stock_portion(prev_pct_change, group_industry_perform, temp_portfolio_stock, number_of_stock):
    #Điều chỉnh tỉ trọng tăng giảm
    if len(temp_portfolio_stock) != (number_of_stock - 1):
        if number_of_stock == 6:
            if group_industry_perform == 'hsA':
                return 13/75 * (1 + prev_pct_change/2)
            elif group_industry_perform == 'hsBCD':
                return 12/75 * (1 + prev_pct_change/2)
        elif number_of_stock == 10:
            if group_industry_perform == 'hsA':
                return 0.12 * (1 + prev_pct_change/2)
            elif group_industry_perform == 'hsBCD':
                return 0.08 * (1 + prev_pct_change/2)
    elif len(temp_portfolio_stock) == (number_of_stock - 1):
        current_total_portion = sum(0 if stock_info.get('portion') is None else 
                                    stock_info.get('portion', 0) for stock_info in temp_portfolio_stock.values())
        return 1 - current_total_portion

#Hàm này để đếm số lượng cổ phiéu đang nắm giữ ở từng nhóm
def stock_count_portfolio(portfolio_count_dict):
    # Đếm số lượng trong hsA
    hsA_dict = portfolio_count_dict.get('hsA', {})
    hsA_count = sum(hsA_dict.values())
    
    # Đếm số lượng trong hsBC
    hsBC_dict = portfolio_count_dict.get('hsBCD', {})
    hsBC_count = sum(hsBC_dict.values())
    
    # Tổng số lượng trong toàn dict
    total_count = hsA_count + hsBC_count
    
    return {'total': total_count, 'hsA': hsA_count, 'hsBCD': hsBC_count}

#Hàm này để cập nhật temp_portfolio_count cho ngày mới bằng những cổ phiếu đang nắm giữ từ hôm trước
def update_portfolio_portion_count(temp_portfolio_stock):
    temp_portfolio_count = {
        'hsA': {  # Nhóm ngành tăng trưởng cao
            'bds': 0, 'xd': 0,'chung_khoan': 0,'tai_chinh': 0,'thep': 0,'ban_le': 0,
        },
        'hsBCD': {  # Nhóm ngành tăng trưởng thấp/trung bình
            'thuy_san': 0, 'det_may': 0, 'cong_nghiep': 0, 'hoa_chat': 0,
            'khoang_san': 0, 'dau_khi': 0, 'bds_kcn': 0, 'cong_nghe': 0,
            'ngan_hang': 0, 'van_tai': 0, 'thuc_pham': 0, 'htd': 0,
            'y_te': 0, 'dulich_dv': 0, 'dv_hatang': 0, 'bao_hiem': 0,
        }
    }

    if len(temp_portfolio_stock) > 0:
        for item in temp_portfolio_stock.values():
            if item['industry_name'] in temp_portfolio_count['hsA'].keys():
                temp_portfolio_count['hsA'][item['industry_name']] += 1
            else:
                temp_portfolio_count['hsBCD'][item['industry_name']] += 1 
                
    return temp_portfolio_count

In [53]:
#Hàm này chọn cổ phiếu để mua mới
def picking_stock(df, temp_portfolio_count, number_of_total_stock, number_of_group_stock, temp_portfolio_stock_count, classification_df):
    df = df.sort_values('t5_score', ascending=False)
    buy_stock_list = []

    for ticker in df['ticker'].tolist():
        # Check điều kiện xem tổng số lượng cổ phiếu đã đủ chưa
        if stock_count_portfolio(temp_portfolio_count)['total'] < number_of_total_stock:

            temp_industry_name = classification_df.set_index('ticker').loc[ticker, 'industry_name']
            temp_industry_perform = classification_df.set_index('ticker').loc[ticker, 'industry_perform']
            group_industry_perform = calculate_group_industry_perform(temp_industry_perform)

            # Check điều kiện xem nhóm hiêu suất đã đủ cổ phiếu chưa, nếu đủ rồi thì bỏ qua cổ phiếu này
            if not stock_count_portfolio(temp_portfolio_count)[group_industry_perform] \
                < number_of_group_stock[group_industry_perform]:
                    continue

            # Check điều kiện xem ngành đã đủ cổ phiếu chưa, nếu đủ rồi thì bỏ qua cổ phiếu này
            if not temp_portfolio_count[group_industry_perform][temp_industry_name] \
                < temp_portfolio_stock_count[group_industry_perform][temp_industry_name]:
                    continue

            #Thêm các cổ phiếu mua với vào list để lưu vào cột buy signal
            buy_stock_list.append(ticker)

            #Tăng các biến đếm số lượng cổ phiếu
            temp_portfolio_count[group_industry_perform][temp_industry_name] += 1

    return buy_stock_list

In [54]:
def technical_filter_df(df, date_key, portfolio_stock_dict, raw_stock_list, marketcap_template):

    # Ngưỡng xác định xu hướng thị trường
    index_week_threshold = -0.01  # Ngưỡng chỉ số tuần
    index_month_threshold = -0.02  # Ngưỡng chỉ số tháng

    # Cấu hình giá trị vốn hóa tối thiểu cho từng loại
    marketcap_template_dict = {
        'small': {'min_value': 500000,'mean_value': 500000},
        'medium': {'min_value': 2000000,'mean_value': 2000000},
        'large': {'min_value': 10000000,'mean_value': 10000000}   
    }

    # Thu thập dữ liệu cổ phiếu tại ngày giao dịch cần kiểm tra
    frames = []
    for ticker in raw_stock_list:
        stock_data = portfolio_stock_dict[ticker]
        day_data = stock_data[stock_data['date'] == date_key]
        if not day_data.empty:
            frames.append(day_data.copy())
            
    # Kiểm tra nếu không có dữ liệu nào
    if not frames:
        return pd.DataFrame()
    
    # Gộp tất cả dữ liệu cổ phiếu vào một DataFrame
    raw_stock_df = pd.concat(frames, ignore_index=True)

    # Lọc cổ phiếu theo điều kiện thanh khoản và vốn hóa
    template_limits = marketcap_template_dict[marketcap_template]
    filtered_stock_df = raw_stock_df[
        (raw_stock_df['t5_score'] > -1) &
        # Thanh khoản tối thiểu và trung bình phải > 50,000
        (raw_stock_df['min_vol'] > 50000) &
        (raw_stock_df['mean_vol'] > 50000) &
        # Vốn hóa phải lớn hơn ngưỡng theo loại
        (raw_stock_df['min_value'] > template_limits['min_value']) & 
        (raw_stock_df['mean_value'] > template_limits['mean_value'])
    ]

    # Lấy thông tin xu hướng thị trường tại ngày giao dịch đang xét
    market_info = df.loc[df['date'] == date_key].iloc[0]
    trend_5p = market_info['trend_5p']    # Xu hướng ngắn hạn (5 phiên)
    trend_20p = market_info['trend_20p']  # Xu hướng trung hạn (20 phiên)

    shift1_5p = market_info['5p_shift1']
    shift1_20p = market_info['20p_shift1']
    
    shift5_5p = market_info['5p_shift5']
    shift5_20p = market_info['20p_shift5']

    diff1_5p = trend_5p - shift1_5p
    diff1_20p = trend_20p - shift1_20p

    diff5_5p = trend_5p - shift5_5p
    diff5_20p = trend_20p - shift5_20p

    # Điều kiện biến động giá (diff_10 phải nằm trong khoảng 0% đến 10%)
    price_change_condition = (filtered_stock_df['diff_lowest10'] < 0.1) & (filtered_stock_df['diff_lowest10'] > 0)

    if marketcap_template in ['small', 'medium']: # Cổ phiếu vốn hóa nhỏ và vừa

        #Lọc điều kiện các giai đoạn sóng đi vào vùng quá cao hoặc quá thấp bỏ các cổ phiếu rủi ro cao
        upper_limit_5p = 0.9
        upper_limit_20p = 0.8
        lower_limit_5p = 0.2
        lower_limit_20p = 0.2
        if (
            (((shift1_5p > upper_limit_5p and trend_5p > upper_limit_5p) or ((shift1_5p < upper_limit_5p and trend_5p > upper_limit_5p) and (diff1_5p*0.1 < trend_5p - upper_limit_5p))) 
            or ((shift1_20p > upper_limit_20p and trend_20p > upper_limit_20p) or ((shift1_20p < upper_limit_20p and trend_20p > upper_limit_20p) and (diff1_20p*0.1 < trend_20p - upper_limit_20p)))) 
        or
            (((shift1_5p < lower_limit_5p and trend_5p < lower_limit_5p) or ((shift1_5p > lower_limit_5p and trend_5p < lower_limit_5p) and (-diff1_5p*0.1 < lower_limit_5p - trend_5p))) 
            or ((shift1_20p < lower_limit_20p and trend_20p < lower_limit_20p) or ((shift1_20p > lower_limit_20p and trend_20p < lower_limit_20p) and (-diff1_20p*0.1 < lower_limit_20p - trend_20p))))
        ):  
            filtered_stock_df = filtered_stock_df[price_change_condition]

        #Lọc điều kiện khi sóng đi vào vùng tiêu cực thì bỏ toàn bộ cổ phiếu
        threshould_5p = 0.4
        threshould_20p = 0.3
        if (
            (((shift1_5p < threshould_5p and trend_5p < threshould_5p) or ((shift1_5p > threshould_5p and trend_5p < threshould_5p) and (-diff1_5p*0.1 < threshould_5p - trend_5p))) and
            ((shift1_20p < threshould_20p and trend_20p < threshould_20p) or ((shift1_20p > threshould_20p and trend_20p < threshould_20p) and (-diff1_20p*0.1 < threshould_20p - trend_20p))))
        and (diff1_5p < 0 and diff1_20p < 0) 
        and (diff5_5p < 0 and diff5_20p < 0)
        ):
            filtered_stock_df = filtered_stock_df.iloc[0:0]

    elif marketcap_template in ['large']: # Cổ phiếu vốn lớn
        
        #Lọc điều kiện các giai đoạn sóng đi vào vùng quá cao hoặc quá thấp bỏ các cổ phiếu rủi ro cao
        upper_limit_5p = 0.78
        upper_limit_20p = 0.68
        lower_limit_5p = 0.42
        lower_limit_20p = 0.32
        if (
            (((shift1_5p > upper_limit_5p and trend_5p > upper_limit_5p) or ((shift1_5p < upper_limit_5p and trend_5p > upper_limit_5p) and (diff1_5p*0.1 < trend_5p - upper_limit_5p))) 
            and ((shift1_20p > upper_limit_20p and trend_20p > upper_limit_20p) or ((shift1_20p < upper_limit_20p and trend_20p > upper_limit_20p) and (diff1_20p*0.1 < trend_20p - upper_limit_20p)))) 
        or
            (((shift1_5p < lower_limit_5p and trend_5p < lower_limit_5p) or ((shift1_5p > lower_limit_5p and trend_5p < lower_limit_5p) and (-diff1_5p*0.1 < lower_limit_5p - trend_5p))) 
            and ((shift1_20p < lower_limit_20p and trend_20p < lower_limit_20p) or ((shift1_20p > lower_limit_20p and trend_20p < lower_limit_20p) and (-diff1_20p*0.1 < lower_limit_20p - trend_20p))))
        ):  
            filtered_stock_df = filtered_stock_df[price_change_condition]

        #Lọc điều kiện khi sóng đi vào vùng tiêu cực thì bỏ toàn bộ cổ phiếu
        threshould_5p = 0.3
        threshould_20p = 0.6
        if (
            (((shift1_5p < threshould_5p and trend_5p < threshould_5p) or ((shift1_5p > threshould_5p and trend_5p < threshould_5p) and (-diff1_5p*0.1 < threshould_5p - trend_5p))) and
            ((shift1_20p < threshould_20p and trend_20p < threshould_20p) or ((shift1_20p > threshould_20p and trend_20p < threshould_20p) and (-diff1_20p*0.1 < threshould_20p - trend_20p))))
        and (diff1_5p < 0 and diff1_20p < 0) 
        and (diff5_5p < 0 and diff5_20p < 0)
        ):
            filtered_stock_df = filtered_stock_df.iloc[0:0]

    # Kiểm tra xu hướng thị trường để áp dụng lọc theo ngưỡng Fibonacci
    # Nếu thị trường không giảm quá nhiều trong tuần (5 phiên)
    if market_info['return_cumsum5'] > index_week_threshold:
        # Giá đóng cửa phải cao hơn mức Fibonacci 61.8% theo tuần
        filtered_stock_df = filtered_stock_df[filtered_stock_df['close'] > filtered_stock_df['WFIBO_0618']]
    
    # Nếu thị trường không giảm quá nhiều trong tháng (20 phiên)
    if market_info['return_cumsum20'] > index_month_threshold:
        # Giá đóng cửa phải cao hơn mức Fibonacci 61.8% theo tháng
        filtered_stock_df = filtered_stock_df[filtered_stock_df['close'] > filtered_stock_df['MFIBO_0618']]

    return filtered_stock_df

In [55]:
def selling_stock(temp_portfolio_stock, marketcap_template, date_key, portfolio_stock_dict):
    sell_stock_df = pd.DataFrame()
    for ticker in temp_portfolio_stock.keys():
        temp_df = portfolio_stock_dict[ticker].copy()
        temp_df = temp_df[temp_df['date'] == date_key]
        price_range = (temp_df['high'].item() - temp_df['close'].item())/temp_df['close'].item()
            
        if marketcap_template in ['small', 'medium']:
            temp_df = temp_df[
                    (temp_df['diff_highest10'] < -0.1) |
                    ((temp_df['t5_score'] < -0.1) & 
                        (temp_df['close'] < temp_df['WFIBO_0382']) & 
                        ((temp_df['close'] > temp_df['WFIBO_0618']) | ((temp_df['high'] > temp_df['WFIBO_0618']) & (price_range < 0.01)))
            )]
        elif marketcap_template in ['large']:
            temp_df = temp_df[
                    ((temp_df['t5_score'] < -0.1) & 
                        (temp_df['close'] < temp_df['WFIBO_0382']) & 
                        ((temp_df['close'] > temp_df['WFIBO_0618']) | ((temp_df['high'] > temp_df['WFIBO_0618']) & (price_range < 0.01)))
            )]          
                    
        sell_stock_df = pd.concat([sell_stock_df, temp_df], axis=0)

    # Chỉ bán các cổ phiếu đã nắm giữ ít nhất 2 ngày (đủ điều kiện T+)
    sell_stock_list = []
    if len(sell_stock_df) > 0:
        for ticker in sell_stock_df['ticker'].tolist():
            if temp_portfolio_stock[ticker]['days_held'] >= 2:
                sell_stock_list.append(ticker)

    return sell_stock_list

In [56]:
def auto_portfolio(df, portfolio_stock_dict, classification_df, portfolio_template, marketcap_template):
    """
    Hàm tự động xây dựng và quản lý danh mục đầu tư theo thời gian
    
    Args:
        df: DataFrame chỉ số thị trường chứa các tín hiệu giao dịch
        portfolio_stock_dict: Dictionary chứa dữ liệu cổ phiếu theo mã
        classification_df: DataFrame phân loại ngành nghề của các cổ phiếu
        portfolio_template: Loại danh mục ('6stocks', '10stocks', '20stocks')
        marketcap_template: Loại vốn hóa ('small', 'medium', 'large')
    
    Returns:
        df: DataFrame đã cập nhật tín hiệu mua/bán
        holding_stock_dict: Dictionary theo dõi các cổ phiếu nắm giữ theo thời gian
    """
    df = df.copy()

    # Cấu hình số lượng cổ phiếu phân bổ theo ngành cho từng loại danh mục
    portfolio_stock_count_dict = {
        '6stocks': {
            'hsA': {  # Nhóm ngành tăng trưởng cao
                'bds': 2, 'xd': 2,'chung_khoan': 2,'tai_chinh': 1,'thep': 1,'ban_le': 1,
            },
            'hsBCD': {  # Nhóm ngành tăng trưởng thấp/trung bình
                'thuy_san': 1, 'det_may': 1, 'cong_nghiep': 1, 'hoa_chat': 1,
                'khoang_san': 1, 'dau_khi': 1, 'bds_kcn': 1, 'cong_nghe': 1,
                'ngan_hang': 1, 'van_tai': 1, 'thuc_pham': 1, 'htd': 1,
                'y_te': 1, 'dulich_dv': 1, 'dv_hatang': 1, 'bao_hiem': 1,
            }
        },
        '10stocks': {
            'hsA': {
                'bds': 3, 'xd': 2, 'chung_khoan': 3, 'tai_chinh': 1, 'thep': 2, 'ban_le': 1,
            },
            'hsBCD': {
                'thuy_san': 1, 'det_may': 1, 'cong_nghiep': 1, 'hoa_chat': 1,
                'khoang_san': 1, 'dau_khi': 1, 'bds_kcn': 1, 'cong_nghe': 1,
                'ngan_hang': 1, 'van_tai': 1, 'thuc_pham': 1, 'htd': 1,
                'y_te': 1, 'dulich_dv': 1, 'dv_hatang': 1, 'bao_hiem': 1,
            }
        },
    }


    # Lấy cấu hình cho loại danh mục đã chọn
    temp_portfolio_stock_count = portfolio_stock_count_dict[portfolio_template]

    # Thiết lập số lượng cổ phiếu tối đa và phân bổ theo nhóm
    if portfolio_template == '6stocks':
        number_of_total_stock = 6
        number_of_group_stock = {'hsA': 3, 'hsBCD': 3}
    elif portfolio_template == '10stocks':
        number_of_total_stock = 10
        number_of_group_stock = {'hsA': 5, 'hsBCD': 5}

    # Dictionary lưu trữ các cổ phiếu nắm giữ theo từng ngày
    holding_stock_dict = {}

    # Duyệt từ cuối lên đầu danh sách để xây dựng danh mục theo thời gian
    for i in range(len(df) - 1, -1, -1):
        date_key = df.at[i, 'date'].strftime('%Y-%m-%d')  # Ngày hiện tại

        # --- XỬ LÝ NGÀY ĐẦU TIÊN ---
        if i == len(df) - 1:
            # Nếu ngày đầu tiên có tín hiệu mua mới (phase=1, phase_s1=0, phase_s2=0)
            if ((df.at[i, 'phase'] == 1) & (df.at[i, 'phase_s1'] == 0) & (df.at[i,'phase_s2'] == 0)):
                # Khởi tạo danh mục trống
                temp_portfolio_count = update_portfolio_portion_count({})
                raw_stock_list = df.at[i, 'raw_list']
                # Lọc cổ phiếu theo tiêu chí kỹ thuật
                filtered_stock_df = technical_filter_df(df, date_key, portfolio_stock_dict, raw_stock_list, marketcap_template)
                df.at[i, 'filtered_list'] = list(filtered_stock_df['ticker'].tolist())
                # Chọn cổ phiếu để mua
                df.at[i, 'buy_signal'] = picking_stock(filtered_stock_df, temp_portfolio_count, 
                                                      number_of_total_stock, number_of_group_stock, 
                                                      temp_portfolio_stock_count, classification_df)

            # Ngày đầu tiên chưa có cổ phiếu nắm giữ
            holding_stock_dict[date_key] = {}
            continue

        # --- XỬ LÝ CÁC NGÀY TIẾP THEO ---
        # Lấy thông tin ngày trước đó
        prev_date_key = df.at[i+1, 'date'].strftime('%Y-%m-%d')
        
        # Lọc và cập nhật danh mục từ ngày trước (loại bỏ các cổ phiếu đã có tín hiệu bán)
        temp_portfolio_stock = {
            key: copy.deepcopy(value) 
            for key, value in holding_stock_dict[prev_date_key].items()
            if key not in df.at[i+1, 'sell_signal']
        }
        
        # Tăng số ngày nắm giữ cho các cổ phiếu còn lại
        for ticker in temp_portfolio_stock.keys():
            temp_portfolio_stock[ticker]['days_held'] += 1

        # --- THÊM CỔ PHIẾU MỚI VÀO DANH MỤC ---
        # Xử lý các cổ phiếu đã có tín hiệu mua ở phiên trước
        for ticker in df.at[i+1, 'buy_signal']:
            # Lấy thông tin ngành nghề của cổ phiếu
            temp_industry_name = classification_df.set_index('ticker').loc[ticker, 'industry_name']
            temp_industry_perform = classification_df.set_index('ticker').loc[ticker, 'industry_perform']
            group_industry_perform = calculate_group_industry_perform(temp_industry_perform)

            # Lấy giá mở cửa và giá tham chiếu
            open_price = portfolio_stock_dict[ticker].set_index('date').loc[date_key, 'open']
            trade_price = portfolio_stock_dict[ticker].set_index('date').loc[date_key, 'trade_price']
            prev_close_price = portfolio_stock_dict[ticker].set_index('date').loc[prev_date_key, 'close']
            prev_pct_change = portfolio_stock_dict[ticker].set_index('date').loc[prev_date_key, 'pct_change']

            # Chỉ mua cổ phiếu khi giá mở cửa không tăng quá 2% so với giá tham chiếu
            if (((open_price - prev_close_price)/prev_close_price) < 0.02):
                temp_portfolio_stock[ticker] = {
                    'buy_price': trade_price,
                    'buy_date': date_key,
                    'industry_name': temp_industry_name,
                    'industry_perform': temp_industry_perform,
                    'days_held': 0,  # Mới mua nên days_held = 0
                    'portion': calculate_stock_portion(prev_pct_change, group_industry_perform, temp_portfolio_stock, number_of_total_stock)
                }
        
        # --- CẬP NHẬT THÔNG TIN DANH MỤC HIỆN TẠI ---
        # Tính toán số lượng cổ phiếu theo nhóm ngành
        temp_portfolio_count = update_portfolio_portion_count(temp_portfolio_stock)
        # Lưu danh mục ngày hiện tại
        holding_stock_dict[date_key] = copy.deepcopy(temp_portfolio_stock)
        # Cập nhật danh sách cổ phiếu đang nắm giữ vào DataFrame
        df.at[i, 'holding_list'] = list(temp_portfolio_stock.keys())

        # --- LỌC CỔ PHIẾU TIỀM NĂNG CHO PHIÊN HIỆN TẠI ---
        raw_stock_list = df.at[i, 'raw_list']
        filtered_stock_df = technical_filter_df(df, date_key, portfolio_stock_dict, raw_stock_list, marketcap_template)

        # Loại bỏ các cổ phiếu đã nắm giữ hoặc đã có tín hiệu mua
        if len(filtered_stock_df) > 0: 
            filtered_stock_df = filtered_stock_df[~filtered_stock_df['ticker'].isin(holding_stock_dict[prev_date_key].keys())]
            filtered_stock_df = filtered_stock_df[~filtered_stock_df['ticker'].isin(df.at[i+1, 'buy_signal'])]

            # Cập nhật danh sách cổ phiếu sau khi lọc qua điều kiện kĩ thuật và DF
            df.at[i, 'filtered_list'] = list(filtered_stock_df['ticker'].tolist())
        
        # --- XỬ LÝ THEO TỪNG GIAI ĐOẠN CỦA CHU KỲ GIAO DỊCH ---
        # Ngày đầu tiên của chu kỳ mới (phase=1, phase_s1=0, phase_s2=0)
        if ((df.at[i, 'phase'] == 1) & (df.at[i, 'phase_s1'] == 0) & (df.at[i,'phase_s2'] == 0)):
            # Chọn cổ phiếu mua mới
            df.at[i, 'buy_signal'] = picking_stock(filtered_stock_df, temp_portfolio_count, 
                                                 number_of_total_stock, number_of_group_stock, 
                                                 temp_portfolio_stock_count, classification_df)
        
        # Ngày thứ hai của chu kỳ mới (phase=1, phase_s1=1, phase_s2=0)
        elif ((df.at[i, 'phase'] == 1) & (df.at[i, 'phase_s1'] == 1) & (df.at[i,'phase_s2'] == 0)):
            # Không cần làm gì, chỉ mua cổ phiếu theo tín hiệu phiên trước
            pass

        # Giai đoạn chính của chu kỳ giao dịch (phase=1, phase_s1=1, phase_s2=1)
        elif ((df.at[i, 'phase'] == 1) & (df.at[i, 'phase_s1'] == 1) & (df.at[i,'phase_s2'] == 1)):                
            # Cập nhật tín hiệu bán
            df.at[i, 'sell_signal'] = selling_stock(temp_portfolio_stock, marketcap_template, date_key, portfolio_stock_dict)
            
            # Chọn cổ phiếu mới để thay thế các mã đã bán
            df.at[i, 'buy_signal'] = picking_stock(filtered_stock_df, temp_portfolio_count, 
                                                 number_of_total_stock, number_of_group_stock, 
                                                 temp_portfolio_stock_count, classification_df)
            
        # Các giai đoạn khác (tín hiệu bán hoặc kết thúc chu kỳ)
        else:
            sell_stock_list = []
            for ticker in temp_portfolio_stock.keys():
                if ticker in holding_stock_dict[date_key].keys():
                    if temp_portfolio_stock[ticker]['days_held'] >= 2:
                        sell_stock_list.append(ticker)

            df.at[i, 'sell_signal'] = list(sell_stock_list)
        
    # Tính số lượng cổ phiếu trong danh mục theo ngày
    df['number_of_stock'] = df['holding_list'].apply(lambda x: len(x))
    
    return df, holding_stock_dict

In [57]:
def calculate_stock_performance(portfolio_df, holding_dict, portfolio_stock_dict):
    """
    Tính hiệu suất của các cổ phiếu trong danh mục đầu tư với tối ưu tốc độ.
    
    Args:
        portfolio_df: DataFrame danh mục đầu tư
        holding_dict: Dictionary chứa cổ phiếu nắm giữ theo ngày
        portfolio_stock_dict: Dictionary chứa dữ liệu gốc của cổ phiếu
        date_series: Series ngày (tùy chọn, sẽ tạo từ portfolio_df nếu không cung cấp)
    
    Returns:
        Dictionary chứa DataFrame hiệu suất cho từng mã cổ phiếu
    """
    
    # Bước 1: Xác định tất cả cổ phiếu từng nằm trong danh mục để tránh lặp lại
    all_portfolio_stocks = set()
    for stocks in portfolio_df['holding_list']:
        all_portfolio_stocks.update(stocks)
    all_portfolio_stocks = sorted(list(all_portfolio_stocks))
    
    # Bước 2: Tạo DataFrame theo dõi tỷ trọng cổ phiếu theo ngày
    stock_phase_df = portfolio_df[['date']]
    date_keys = [date.strftime('%Y-%m-%d') for date in stock_phase_df['date']]
    
    # Tạo dict ánh xạ từ date_key sang index để tra cứu nhanh
    date_to_idx = {date_key: idx for idx, date_key in enumerate(date_keys)}
    
    # Bước 3: Tạo ndarray cho tỷ trọng - nhanh hơn việc cập nhật DataFrame
    num_dates = len(stock_phase_df)
    portion_matrix = np.zeros((num_dates, len(all_portfolio_stocks)))
    
    # Lưu index của mỗi ticker để truy cập nhanh vào ma trận
    ticker_to_col_idx = {ticker: idx for idx, ticker in enumerate(all_portfolio_stocks)}
    
    # Điền tỷ trọng vào ma trận
    for date_key, holdings in holding_dict.items():
        if date_key in date_to_idx:
            row_idx = date_to_idx[date_key]
            for ticker, info in holdings.items():
                if ticker in ticker_to_col_idx:
                    col_idx = ticker_to_col_idx[ticker]
                    portion_matrix[row_idx, col_idx] = info['portion']
    
    # Bước 4: Chuyển ma trận tỷ trọng vào DataFrame
    for i, ticker in enumerate(all_portfolio_stocks):
        stock_phase_df[ticker] = portion_matrix[:, i]
    
    # Bước 5: Tính toán hiệu suất cho từng mã cổ phiếu
    stock_perform_dict = {}
    
    # Tối ưu loop chính - xử lý từng cổ phiếu
    for ticker, df in portfolio_stock_dict.items():
        # Chỉ trích xuất các cột cần thiết
        temp_df = df[['date', 'open', 'close','trade_price']].copy()
        
        # Thêm cột portion
        if ticker in all_portfolio_stocks:
            portions = stock_phase_df[ticker].values.copy()  # Sử dụng .copy() để tránh thay đổi giá trị gốc
        else:
            portions = np.zeros(len(temp_df))
            
        temp_df['portion'] = portions
            
        # Tính phase và phase_s1
        phase = (temp_df['portion'] > 0).astype(int).values
        phase_s1 = np.pad(phase[1:], (0, 1), 'constant')
        phase_s2 = np.pad(phase[2:], (0, 2), 'constant')
        
        # Chuẩn bị arrays cho tính toán return nhanh
        n = len(temp_df)
        returns = np.zeros(n)
        modified_portions = portions.copy()
        
        # Lấy các arrays để tính toán vector hóa khi có thể
        closes = temp_df['close'].values
        trade_prices = temp_df['trade_price'].values

        # Vòng lặp kết hợp xử lý portion và returns
        for i in range(n-2, -1, -1):
            # Xử lý cập nhật portion cho các pattern đặc biệt
            if phase[i] == 0 and phase_s1[i] == 1 and phase_s2[i] == 1 and i+1 < n:
                modified_portions[i] = portions[i+1]
                
            # Tính toán returns dựa trên các pattern
            if phase[i] == 1 and phase_s1[i] == 1 and phase_s2[i] == 0:
                # Trường hợp mua
                returns[i] = (closes[i] - trade_prices[i]) / trade_prices[i]
            elif phase[i] == 1 and phase_s1[i] == 1 and phase_s2[i] == 1:
                # Trường hợp tiếp tục nắm giữ
                returns[i] = (closes[i] - closes[i+1]) / closes[i+1]
            elif phase[i] == 0 and phase_s1[i] == 1 and phase_s2[i] == 1:
                # Trường hợp ngày đầu tiên có tín hiệu bán
                returns[i] = (closes[i] - closes[i+1]) / closes[i+1]
            elif phase[i] == 0 and phase_s1[i] == 0 and phase_s2[i] == 1:
                # Trường hợp bán sau khi có tín hiệu
                returns[i] = (trade_prices[i] - closes[i+1]) / closes[i+1]
            # Else returns[i] đã là 0 rồi
        
        # Gán kết quả trở lại DataFrame
        temp_df['phase'] = phase
        temp_df['phase_s1'] = phase_s1
        temp_df['phase_s2'] = phase_s2
        temp_df['portion'] = modified_portions
        temp_df['return'] = returns
        temp_df['portion_return'] = temp_df['return'] * temp_df['portion']
        
        stock_perform_dict[ticker] = temp_df
    
    return stock_perform_dict

In [58]:
def extract_trade_history(stock_perform_dict):
    """
    Trích xuất lịch sử giao dịch từ dictionary hiệu suất cổ phiếu.
    
    Args:
        stock_perform_dict: Dictionary chứa DataFrame hiệu suất cho mỗi mã cổ phiếu
        
    Returns:
        DataFrame chứa lịch sử giao dịch với các cột: ticker, buy_date, sell_date, buy_price, sell_price, 
        portion, return, day_count (số ngày nắm giữ)
    """
    # Khởi tạo DataFrame rỗng để lưu kết quả
    trade_history_df = pd.DataFrame()

    # Duyệt qua từng cổ phiếu trong dictionary
    for ticker, df in stock_perform_dict.items():
        # Tạo bản sao để không ảnh hưởng dữ liệu gốc
        temp_df = df.copy().reset_index(drop=True)
        
        # Khởi tạo danh sách lưu các chỉ số của ngày mua và bán
        trading_pairs = []
        
        # Đánh dấu bắt đầu của một giai đoạn mua
        start_idx = None
        
        # Duyệt từ dưới lên (từ cũ đến mới) theo đúng thứ tự thời gian
        for i in range(len(temp_df)-1, -1, -1):
            # Tìm điểm bắt đầu mua (chuyển từ 0 sang 1)
            if i < len(temp_df)-1 and temp_df['phase'].iloc[i+1] == 0 and temp_df['phase'].iloc[i] == 1:
                start_idx = i
            
            # Tìm điểm kết thúc mua (chuyển từ 1 sang 0) và đã có điểm bắt đầu
            elif start_idx is not None and i < len(temp_df)-1 and temp_df['phase'].iloc[i+1] == 1 and temp_df['phase'].iloc[i] == 0:
                end_idx = i
                # Lưu cặp mua-bán vào danh sách với thứ tự đúng
                trading_pairs.append((start_idx, end_idx))
                start_idx = None
        
        # Trường hợp đặc biệt: nếu phase kết thúc với 1 mà chưa có điểm bán
        if start_idx is not None and temp_df['phase'].iloc[0] == 1:
            trading_pairs.append((start_idx, 0))
        
        # Tạo DataFrame mới để lưu thông tin giao dịch
        if trading_pairs:
            trade_data = {
                'buy_date': [temp_df.at[buy_idx, 'date'] for buy_idx, _ in trading_pairs],
                'sell_date': [temp_df.at[sell_idx, 'date'] for _, sell_idx in trading_pairs],
                'buy_price': [temp_df.at[buy_idx, 'trade_price'] for buy_idx, _ in trading_pairs],
                'sell_price': [temp_df.at[sell_idx, 'trade_price'] for _, sell_idx in trading_pairs],
                'portion': [temp_df.at[buy_idx, 'portion'] for buy_idx, _ in trading_pairs],
                'days_held': [buy_idx - sell_idx for buy_idx, sell_idx in trading_pairs]
            }
            
            ticker_df = pd.DataFrame(trade_data)
            ticker_df['ticker'] = ticker
            ticker_df['return'] = (ticker_df['sell_price'] - ticker_df['buy_price']) / ticker_df['buy_price']
            
            trade_history_df = pd.concat([trade_history_df, ticker_df], axis=0)
    
    return trade_history_df

In [59]:
def calculate_stock_mean_return(trade_history_df, portfolio_name):
    """
    Tóm tắt hiệu suất giao dịch cho từng mã cổ phiếu
    
    Args:
        trade_history_df: DataFrame chứa lịch sử giao dịch
        
    Returns:
        DataFrame với thông tin hiệu suất trung bình và số lượt mua cho mỗi mã
    """
    if trade_history_df.empty:
        return pd.DataFrame(columns=['ticker', 'mean_return', 'buy_count'])
    
    # Sử dụng groupby để tính toán hiệu quả hơn
    result_df = trade_history_df.groupby('ticker').agg(
        mean_return=('return', 'mean'),
        buy_count=('return', 'count')
    ).reset_index()

    result_df['name'] = portfolio_name
    
    return result_df

In [60]:
def calculate_portfolio_return(date_series, stock_perform_dict):
    portfolio_return_df = date_series.copy()
    for ticker, df in stock_perform_dict.items():
        portfolio_return_df[ticker] = df['portion_return'].fillna(0)

    portfolio_return_df['sum_return'] = portfolio_return_df.iloc[:,1:].sum(axis=1)

    return portfolio_return_df['sum_return']

In [61]:
# Hàm tạo lịch sử giao dịch cho từng ngày để xem mỗi ngày mua gì bán gì kèm tỉ trọng và mức giá
def expand_signals(df):
    rows = []
    for index, row in df.iterrows():
        date = row['date']
        if isinstance(row['buy_signal'], list):
            buy_signals = row['buy_signal']
        else:
            buy_signals_str = row['buy_signal'].strip('[]')
            buy_signals = [ticker.strip() for ticker in buy_signals_str.split(',')] if buy_signals_str else []
        if isinstance(row['sell_signal'], list):
            sell_signals = row['sell_signal']
        else:
            sell_signals_str = row['sell_signal'].strip('[]')
            sell_signals = [ticker.strip() for ticker in sell_signals_str.split(',')] if sell_signals_str else []
        for ticker in buy_signals:
            if ticker:  # Bỏ qua các ticker rỗng
                rows.append({'date': date,'ticker': ticker,'type': 'buy'})
        for ticker in sell_signals:
            if ticker:  # Bỏ qua các ticker rỗng
                rows.append({'date': date,'ticker': ticker,'type': 'sell'})
    # Tạo DataFrame mới
    return pd.DataFrame(rows)

##### Tính toán hiệu suất

- Tính bảng portfolio_raw_df để làm gốc tính toán cho từng danh mục

In [62]:
portfolio_raw_df = portfolio_index_df[['date','open','high','low','close']]
portfolio_raw_df['quarter'] = portfolio_raw_df['date'].apply(lambda x: next((key for key, value in quarter_timestamp_map_dict.items() if value[0] <= x <= value[1]), None))

portfolio_raw_df[['phase', 'buy_score_check', 'buy_market_score_check', 'buy_hsA_score_check', 'buy_hsC_score_check',
    'buy_ms_check', 'buy_ms5p_check', 'buy_ms20p_check', 'buy_ms60p_check', 'buy_msvalue_check', 'buy_overprice_check',
    'sell_score_check', 'sell_market_score_check', 'sell_hsA_score_check', 'sell_hsC_score_check',
    'sell_ms_check', 'sell_ms5p_check', 'sell_ms20p_check', 'sell_ms60p_check', 'sell_msvalue_check', 'sell_underprice_check']] \
= market_phase_df[['market_phase_final', 'buy_score_check', 'buy_market_score_check', 'buy_hsA_score_check', 'buy_hsC_score_check',
    'buy_ms_check', 'buy_ms5p_check', 'buy_ms20p_check', 'buy_ms60p_check', 'buy_msvalue_check', 'buy_overprice_check',
    'sell_score_check', 'sell_market_score_check', 'sell_hsA_score_check', 'sell_hsC_score_check',
    'sell_ms_check', 'sell_ms5p_check', 'sell_ms20p_check', 'sell_ms60p_check', 'sell_msvalue_check', 'sell_underprice_check']]

portfolio_raw_df['phase_s1'] = portfolio_raw_df['phase'].shift(-1).fillna(0).astype(int)
portfolio_raw_df['phase_s2'] = portfolio_raw_df['phase'].shift(-2).fillna(0).astype(int)

portfolio_raw_df['raw_return'] = portfolio_raw_df['close'][::-1].pct_change()[::-1].fillna(0)
portfolio_raw_df['index_return'] = calculate_pct_return(portfolio_raw_df, 'phase')

portfolio_raw_df['return_cumsum5'] = portfolio_raw_df['raw_return'][::-1].rolling(window=5, min_periods=1).sum()[::-1]
portfolio_raw_df['return_cumsum20'] = portfolio_raw_df['raw_return'][::-1].rolling(window=20, min_periods=1).sum()[::-1]

portfolio_raw_df[['trend_5p','trend_20p']] = market_phase_df[['trend_5p','trend_20p']]
portfolio_raw_df['5p_shift1'] = portfolio_raw_df['trend_5p'].shift(-1).ffill()
portfolio_raw_df['5p_shift5'] = portfolio_raw_df['trend_5p'][::-1].rolling(window=5, min_periods=1).mean()[::-1].shift(-1).ffill()
portfolio_raw_df['20p_shift1'] = portfolio_raw_df['trend_20p'].shift(-1).ffill()
portfolio_raw_df['20p_shift5'] = portfolio_raw_df['trend_20p'][::-1].rolling(window=5, min_periods=1).mean()[::-1].shift(-1).ffill()

portfolio_raw_df['raw_list'] = portfolio_raw_df.apply(lambda x: signal_quarter_stock_list_dict[x['quarter']] if 
                                                        ((x['phase'] == 1) & (x['phase_s1'] == 0) & (x['phase_s2'] == 0)) |
                                                        ((x['phase'] == 1) & (x['phase_s1'] == 1) & (x['phase_s2'] == 0)) |
                                                        ((x['phase'] == 1) & (x['phase_s1'] == 1) & (x['phase_s2'] == 1)) |
                                                        ((x['phase'] == 0) & (x['phase_s1'] == 1) & (x['phase_s2'] == 1))
                                                    else [], axis=1) #else này bao gồm 100, 000 và 001

portfolio_raw_df['filtered_list'] = portfolio_raw_df.apply(lambda _: [], axis=1)
portfolio_raw_df['holding_list'] = portfolio_raw_df.apply(lambda _: [], axis=1)
portfolio_raw_df['buy_signal'] = portfolio_raw_df.apply(lambda _: [], axis=1)
portfolio_raw_df['sell_signal'] = portfolio_raw_df.apply(lambda _: [], axis=1)

- Tính toán các dict hiệu suất cho từng danh mục

In [63]:
portfolio_list = [
    ('6stocks', 'small'),
    # ('10stocks', 'small'),
    # ('6stocks', 'medium'),
    # ('6stocks', 'large')
]

portfolio_progress_dict = {}
portfolio_holding_dict = {}
portfolio_perform_dict = {}
portfolio_trade_history_dict = {}
portfolio_daily_trade_dict = {}

for porfolio_template, marketcap_template in portfolio_list:
    portfolio_name = f'{marketcap_template}_{porfolio_template}'

    # Tạo temp_portfolio_progress, temp_portfolio_holding và temp_portfolio_perform
    temp_portfolio_progress, temp_portfolio_holding = auto_portfolio(portfolio_raw_df, portfolio_stock_dict, stock_classification_df, porfolio_template, marketcap_template)
    temp_portfolio_perform = calculate_stock_performance(temp_portfolio_progress, temp_portfolio_holding, portfolio_stock_dict)

    # Tạo cột return cho temp_portfolio_progress và gán cho portfolio_progress_dict
    temp_portfolio_progress['portfolio_return'] = calculate_portfolio_return(date_series, temp_portfolio_perform)
    portfolio_progress_dict[portfolio_name] = temp_portfolio_progress

    # Gán temp_portfolio_holding và temp_portfolio_perform cho portfolio_holding_dict và portfolio_perform_dict  
    portfolio_holding_dict[portfolio_name] = temp_portfolio_holding
    portfolio_perform_dict[portfolio_name] = temp_portfolio_perform

    # Tạo lịch sử giao dịch cho từng danh mục và gán cho portfolio_trade_history_dict
    portfolio_trade_history_dict[portfolio_name] = extract_trade_history(temp_portfolio_perform)

    # Tạo daily trading và gán cho portfolio_daily_trade_dict
    value_df = temp_portfolio_progress[['date','buy_signal','sell_signal']]
    value_df['buy_signal'] = value_df['buy_signal'].shift(-1)
    value_df['sell_signal'] = value_df['sell_signal'].shift(-1)
    value_df.at[len(value_df)-1, 'buy_signal'] = []
    value_df.at[len(value_df)-1, 'sell_signal'] = []
    expanded_df = expand_signals(value_df)

    temp_df = pd.DataFrame()
    for i in range(len(expanded_df)):
        date_key = expanded_df.at[i, 'date']
        ticker = expanded_df.at[i, 'ticker']
        temp_stock_df = portfolio_stock_dict[ticker][['date','open','high','low','close','trade_high', 'portion_high', 'trade_low', 'portion_low', 'trade_mean', 'portion_mean', 'trade_price']]
        temp_portion_df = portfolio_perform_dict[portfolio_name][ticker][['date', 'portion']]
        merge_row = expanded_df.iloc[[i]].merge(temp_stock_df[temp_stock_df['date'] == date_key], on='date').merge(temp_portion_df[temp_portion_df['date'] == date_key], on='date')
        temp_df = pd.concat([temp_df, merge_row], axis=0)

    portfolio_daily_trade_dict[portfolio_name] = temp_df[~((temp_df['type'] == 'buy') & (temp_df['portion'] == 0))].sort_values('date', ascending=False).reset_index(drop=True)

- Ghép các dict hiệu suất lại thành bảng thông kê tổng hợp tất cả danh mục

In [64]:
#Bảng quá trình từng danh mục
history_portfolio_progress_df = pd.DataFrame()
for portfolio_name, df in portfolio_progress_dict.items():
    temp_df = df[['date', 'quarter', 'phase', 'raw_return', 'index_return', 'raw_list', 'filtered_list', 'holding_list', 'buy_signal', 'sell_signal', 'number_of_stock', 'portfolio_return',
                  'buy_score_check', 'buy_market_score_check', 'buy_hsA_score_check', 'buy_hsC_score_check',
                  'buy_ms_check', 'buy_ms5p_check', 'buy_ms20p_check', 'buy_ms60p_check', 'buy_msvalue_check', 'buy_overprice_check',
                  'sell_score_check', 'sell_market_score_check', 'sell_hsA_score_check', 'sell_hsC_score_check',
                  'sell_ms_check', 'sell_ms5p_check', 'sell_ms20p_check', 'sell_ms60p_check', 'sell_msvalue_check', 'sell_underprice_check',
                  ]]
    temp_df['portfolio_name'] = portfolio_name
    temp_df = temp_df[temp_df['date'] != today]
    history_portfolio_progress_df = pd.concat([history_portfolio_progress_df, temp_df], axis=0)
history_portfolio_progress_df = history_portfolio_progress_df.sort_values(['portfolio_name', 'date'], ascending=[False, False]).reset_index(drop=True)

#Bảng lịch sử giao dịch từng danh mục
history_portfolio_trading_df = pd.DataFrame()
for portfolio_name, df in portfolio_trade_history_dict.items():
    temp_df = df.copy()
    temp_df['portfolio_name'] = portfolio_name
    temp_df = temp_df[temp_df['sell_date'] != today]
    history_portfolio_trading_df = pd.concat([history_portfolio_trading_df, temp_df], axis=0)
history_portfolio_trading_df = history_portfolio_trading_df.sort_values(['portfolio_name', 'buy_date'], ascending=[False, False]).reset_index(drop=True)

#Bảng trading theo ngày từng danh mục
history_portfolio_order_df = pd.DataFrame()
for portfolio_name, df in portfolio_daily_trade_dict.items():
    temp_df = df.copy()
    temp_df['portfolio_name'] = portfolio_name
    temp_df = temp_df[temp_df['date'] != today]
    history_portfolio_order_df = pd.concat([history_portfolio_order_df, temp_df], axis=0)
history_portfolio_order_df = history_portfolio_order_df.sort_values(['portfolio_name', 'date'], ascending=[False, False]).reset_index(drop=True)

#Bảng cổ phiếu nắm giữ từng danh mục
history_portfolio_holding_df = pd.DataFrame()
for portfolio_name, portfolio_data in portfolio_holding_dict.items():
    for date_key, holdings in portfolio_data.items():
        for ticker, value in holdings.items():
            row = {
                'portfolio_name': portfolio_name,
                'date': pd.to_datetime(date_key),
                'ticker': ticker,
                'close_price': portfolio_stock_dict[ticker].set_index('date').loc[date_key, 'close'],
            }
            for key, val in value.items():
                row[key] = val
            if 'buy_price' in row and row['buy_price'] != 0:
                row['return'] = ((row['close_price'] - row['buy_price']) / row['buy_price'])
            else:
                row['return'] = 0
            history_portfolio_holding_df = pd.concat([history_portfolio_holding_df, pd.DataFrame([row])])

history_portfolio_holding_df = history_portfolio_holding_df[history_portfolio_holding_df['date'] != today]
history_portfolio_holding_df = history_portfolio_holding_df.sort_values(['portfolio_name', 'date', 'buy_date'], ascending=[False, False, False]).reset_index(drop=True)
history_portfolio_holding_df = history_portfolio_holding_df[['date', 'ticker', 'buy_date', 'days_held', 'buy_price', 'close_price', 'return', 'portion', 'industry_name', 'industry_perform', 'portfolio_name']]

In [65]:
# temp_df = history_portfolio_progress_df[history_portfolio_progress_df['portfolio_name'] == 'small_6stocks'].copy()
# temp_df['portfoio_cum_return'] = temp_df['portfolio_return'] [::-1].cumsum()[::-1]
# temp_df['index_cum_return'] = temp_df['index_return'] [::-1].cumsum()[::-1]
# temp_df['raw_cum_return'] = temp_df['raw_return'] [::-1].cumsum()[::-1]
# temp_df.to_excel('../.pbix/test_small_6stocks.xlsx', index=False)

- Dữ liệu cho bảng cổ phiếu lọc và tín hiệu mua bán báo trước

In [66]:
history_portfolio_filtered_df_list = []
for portfolio_name, portfolio_progress_df in portfolio_progress_dict.items():
    for date_key in date_series['date']:
        temp_stock_list = portfolio_progress_df[portfolio_progress_df['date'] == date_key]['filtered_list'].item()
        for stock in temp_stock_list:
            temp_df = portfolio_stock_dict[stock][['ticker', 'date', 'open', 'high', 'low', 'close', 'pct_change', 'vol_ratio', 't5_score']]
            temp_df = temp_df[temp_df['date'] == date_key]
            temp_df['portfolio_name'] = portfolio_name
            history_portfolio_filtered_df_list.append(temp_df)

history_portfolio_filtered_df = pd.concat(history_portfolio_filtered_df_list, axis=0).reset_index(drop=True)
history_portfolio_filtered_df = history_portfolio_filtered_df[history_portfolio_filtered_df['date'] != today]
history_portfolio_filtered_df['industry_name'] = history_portfolio_filtered_df['ticker'].map(stock_classification_df.set_index('ticker')['industry_name'])

In [67]:
history_portfolio_signal_df_list = []
for portfolio_name, portfolio_progress_df in portfolio_progress_dict.items():
    for i, date_key in enumerate(date_series['date']):
        for column in ['buy_signal', 'sell_signal']:
            temp_stock_list = portfolio_progress_df[portfolio_progress_df['date'] == date_key][column].item()
            for stock in temp_stock_list:
                temp_df = portfolio_stock_dict[stock][['ticker', 'date']]
                temp_df = temp_df[temp_df['date'] == date_key]
                temp_df['signal_date'] = date_series.at[i-1, 'date']
                temp_df['type'] = column
                temp_df['portfolio_name'] = portfolio_name
                history_portfolio_signal_df_list.append(temp_df)

history_portfolio_signal_df = pd.concat(history_portfolio_signal_df_list, axis=0).reset_index(drop=True)
history_portfolio_signal_df = history_portfolio_signal_df[history_portfolio_signal_df['date'] != today]
history_portfolio_signal_df['type'] = history_portfolio_signal_df['type'].map({'buy_signal': 'Mua', 'sell_signal': 'Bán'})

#### Lưu dữ liệu

- Bộ thống kế cổ phiếu theo tín hiệu

In [68]:
performance_stock_df_list = []
for ticker, df in portfolio_stock_dict.items():
    if ticker not in prohibited_stock_list:
        temp_df = df[['date','ticker','pct_change']]
        temp_df['phase'] = portfolio_raw_df['phase']
        temp_df['return'] = portfolio_raw_df['phase'] * temp_df['pct_change']
        temp_df['cum_return'] = temp_df['return'][::-1].cumsum()[::-1]
        temp_df['mean_return'] = temp_df['return'].mean()
        temp_df['min_return'] = temp_df['return'].min()
        temp_df['max_return'] = temp_df['return'].max()
        performance_stock_df_list.append(temp_df[['ticker', 'cum_return', 'mean_return', 'min_return', 'max_return']].iloc[[0]])

performance_stock_df = pd.concat(performance_stock_df_list, axis=0).sort_values('cum_return', ascending=False).reset_index(drop=True)
performance_stock_df['industry_name'] = performance_stock_df['ticker'].map(stock_classification_df.set_index('ticker')['industry_name'])
performance_stock_df.to_excel('../.xlsx/performance_stock_statistic.xlsx', index=False)

- Lưu lại current_quarter_signal_list để tính EOD cho Portfolio

In [69]:
# Tìm độ dài lớn nhất trong các danh sách
max_length = max(len(stock_list) for stock_list in signal_quarter_stock_list_dict.values())

# Tạo DataFrame và tự động thêm None cho các danh sách ngắn hơn
signal_quarter_stock_df = pd.DataFrame()
for quarter, stock_list in signal_quarter_stock_list_dict.items():
    # Bổ sung None vào cuối danh sách nếu cần
    padded_list = stock_list + [None] * (max_length - len(stock_list))
    signal_quarter_stock_df[quarter] = padded_list
    
overwrite_mongo(ref_db["signal_quarter_stock"], signal_quarter_stock_df)

- Lưu các bảng history_porfolio

In [70]:
overwrite_mongo(stock_db["history_portfolio_progress"], history_portfolio_progress_df)
overwrite_mongo(stock_db["history_portfolio_trading"], history_portfolio_trading_df)
overwrite_mongo(stock_db["history_portfolio_order"], history_portfolio_order_df)
overwrite_mongo(stock_db["history_portfolio_holding"], history_portfolio_holding_df)
overwrite_mongo(stock_db["history_portfolio_signal"], history_portfolio_signal_df)
overwrite_mongo(stock_db["history_portfolio_filtered"], history_portfolio_filtered_df)